In [2]:
from bertopic import BERTopic
from scipy.cluster import hierarchy as sch
import pandas as pd
import logging
from sklearn.metrics.pairwise import cosine_distances
from tqdm import tqdm
import numpy as np
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
import re
import sys
sys.path.append('../../')
from utility.utility_functions import *

In [3]:
#  '\\\\Cm\\CM\\'

In [ ]:
df = pd.read_parquet('../../data/processed/05_avocado_cleaned_clustered_wip.parquet', engine='pyarrow')

,text
0,Bug 5539\r\nThe screen wizard now ignores the ...
1,"At 02:50 PM 3/15/2001 +0100, it seemed like Ma..."
2,"All,\r\nWe are still in a Nor'easter, but the ..."
3,Responding to Robert Sartin (07:02 AM 6/14/200...
4,I will try to get the real secure up at the fi...
...,...
128640,Checked in WinInetBrowser.java to fix a bug in...
128641,Just wanted to make sure that you know I did t...
128642,"In the end, you will always fail to think of e..."
128643,"Yes, it's valid for Europe, as well. Venk for..."


In [5]:
texts = df['text']

In [9]:
test = texts[texts.str.startswith('\\\\')]
testest = test.tolist()
testest.sort()
testest

['\\\\Cm\\Burn\\4.5GA_Germanium.x\\05-06-2002\\003\\fullinstaller',
 "\\\\Cm\\CM\\Build\\4.0GA_palladium.x\\04-01-2002\\136\\fullinstaller\\setup.exe\r\nWith this build, Full Installer seems working fine, but there are problems.\r\n1. You can not run OCA.  If you try to run OCA, from d:\\build\\bin\\runOCA.bat --> it just doesn't run\r\n2. If you try to write your own batch file to run OCA, since the schema.xml is not there, the Biogen Application won't run\r\n3. The convertutil.jar is not there in the build.  So we can not use Roopak's utility.\r\nSreenivas",
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\05-07-2002\\004\\fullinstaller',
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\06-26-2002\\1090\\fullinstaller\\setup.exe\r\nHi Wilhan,\r\nInstaller fails to install the online.  Do you know how to proceed?',
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\06-27-2002\\037\\fullinstaller',
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\07-08-2002\\039\\fullinstaller',
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\07-2

In [8]:
for t in test:
    print(repr(t))

"\\\\Cm\\CM\\Build\\4.0GA_palladium.x\\04-01-2002\\136\\fullinstaller\\setup.exe\r\nWith this build, Full Installer seems working fine, but there are problems.\r\n1. You can not run OCA.  If you try to run OCA, from d:\\build\\bin\\runOCA.bat --> it just doesn't run\r\n2. If you try to write your own batch file to run OCA, since the schema.xml is not there, the Biogen Application won't run\r\n3. The convertutil.jar is not there in the build.  So we can not use Roopak's utility.\r\nSreenivas"
'\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\10-03-2002\\132\\fullinstaller'
'\\\\Cm\\CM\\Build\\4.6GA_Neon.x\\12-25-2002\\043\\fullinstaller\r\nThe EMDS online tests have passed over this build.\r\nSreenivas'
'\\\\yuyu\\software\\jprobe\r\ncheck sno.txt for license key.\r\n-yuyu'
'\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\10-14-2002\\140\\fullinstaller'
'\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\09-30-2002\\129'
'\\\\Utilitysrv1\\home\\Marketing\\solutions_folder'
'\\\\utilitysrv1\\home\\SalesEngineering\\Accou

In [ ]:
removal_list = [
    '\r\n--------------------------\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)',
    '\r\nDan Baca\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Marketing\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t408.562.8041 w',
    '\r\n---\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Sales Engineering\t\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w',
    '\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Sales Engineering \t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w',
    '\r\n---\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Sales Engineering\t\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w\r\nAvocadoIT, Your Business Everywhere !',
    '\r\n---\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Sales Engineering\t\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w',
    '\r\n---\r\nDan Baca\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Product Marketing\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t408.562.8041 w',
    '--------------------------\r\nFind out how AvocadoIT can mobilize your business (www.avocadoit.com)',
    '\r\n--------------------------\r\nAvocadoIT, Your Business Everywhere!',
    '\r\n-------------------------------------------------------------------\r\n** AvocadoIT, Your Business Everywhere **',
    '\r\n--------------------------\r\nAvocadoIT, Your Business Everywhere! (www.avocadoit.com)',
    '\r\n--------------------------\r\nEmpowered by AvocadoIT Technology (www.avocadoit.com)',
    '************************************\r\nRon D Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel: 408.562.8039\r\nMobile: 415.845.6789\r\nFax:  408.562.8100\r\nrsilverton@avocadoit.com\r\n************************************',
    '\r\n--------------------------\r\nSent to you wirelessly.\r\nAndy Wong\r\n408.562.8131\r\nawong@avocadoit.com',
    '\r\nYour Business  Everywhere\r\n***********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc\r\n2211 N First Street, Suite 200\r\nSan Jose CA, 95131\r\nw 408.562-8039\r\nm 415.845-6789\r\nf 408.562-8100\r\nwww.avocadoit.com\r\n*************************************************',
    '\r\nEnabling Infrastructure for Post-PC Era\r\n************************************\r\nRon D Silverton\r\nDirector of Strategic Marketing\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel: (408) 562-8039\r\nMobile: (415) 845-6789\r\nFax: (408) 408.562.8100\r\nrsilverton@avocadoit.com\r\n************************************',
    '\r\nEnabling Infrastructure for Post-PC Era\r\n************************************\r\nRon D Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131\r\nTel: 408.562.8039\r\nMobile: 415.845.6789\r\nFax:  408.562.8100\r\nrsilverton@avocadoit.com\r\n************************************',
    '\r\n--------------------------\r\nChris Longstaffe\r\n(408) 205 7573\r\nSent using my AvocadoIT wireless device\r\nM-business ... M-powered by AvocadoIT!',
    '\r\nPeter Smialek\r\nRegional Sales Director\r\nAvocadoIT\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile',
    '\r\nAvocadoIT, Your Business Everywhere !',
    '\r\nDirector of Sales, Northeast\r\nAvocadoIT, Inc.\r\n508-893-6095  Office\r\n508-893-0828  Fax\r\n508-740-9126  Mobile\r\njeff.newman@avocadoit.com\r\nwww.avocadoit.com',
    '\r\nCraig Harper\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8175\r\nFax: 408.562.8100\r\ncraig.harper@avocadoit.com\r\nrm -Rf //taliban/bin/laden',
    '\r\n***********************************************************************\r\nRoopak Parikh\r\navocadoit.com\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(408) 588-4057 x17\r\n**********************************************************************',
    '\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!',
    '\r\nSteven Hirata\r\nDirector of Networks and Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8065 \r\nFax: 408.562.8100 \r\nshirata@avocadoit.com \r\nwww.avocadoit.com',
    '\r\n________________________\r\nRayhan bin Abdulmughnee\r\nSystems Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7964\r\nMobile: 408.206-8122\r\nFax: 408.562.8100 \r\nrayhan.abdulmughnee@avocadoit.com \r\nwww.avocadoit.com',
    '\r\nMatt\r\nAvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nMatt DiMaria\r\nSVP, Marketing and Business Development\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8040\r\nwww.avocadoit.com\r\n**********************************************',
    '\r\nChristopher Quinter\r\nDirector of Sales\r\n431 Trumbull Court\r\nNewtown, PA  18940\r\n(267) 757-0736  Office\r\n(215) 778-3235  Mobile\r\nAvocadoIT, Your Business Everywhere! www.avocadoit.com',
    '\r\n---\r\nRicardo Garcia\r\nSales Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 829-4827 cell\r\n(408) 562-7984 office\r\n(408) 562-8100 fax\r\nricardo.garcia@avocadoit.com\r\nwww.avocadoit.com',
    '\r\nBetty Chan-Bauza\r\nGlobal Alliance Manager\r\nAvocadoIT, Inc.\r\nDirect: 1.408.562.8192\r\nFax: 1.408-562.8100\r\nbchan@avocadoit.com\r\nwww.avocadoit.com',
    '\r\n---\r\nGeoff Vona \r\nAvocadoIT\r\nhttp://www.avocadoit.com',
    '\r\n---\r\nRicardo Garcia\r\nSales Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 829-4827 cell\r\n(408) 562-7984 office\r\n(408) 562-8100 fax\r\nricardo.garcia@avocadoit.com\r\nwww.avocadoit.com',
    '\r\nLaura C. Henderson\r\nClient Development\r\nAvocadoIT, Inc.\r\nemail: laura.henderson@avocadoit.com\r\n(800) 355.1103  ext 8187\r\nDirect: (408) 562.8187\r\nCell: (408) 857.8529\r\nhttp://www.avocadoit.com',
    '\r\nAvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nMatt DiMaria\r\nSVP, Marketing and Business Development\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8040\r\nwww.avocadoit.com\r\n**********************************************',
    '\r\nCraig Stewart\r\nVice President\r\nGlobal Telecom Business \r\n____________________________\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, California 95054\r\nMobile:   510-409-7173\r\nOffice:    408-562-8060  \r\nFax:       408-562-8100\r\nEmail:    cstewart@avocadoit.com\r\nWeb:      www.avocadoit.com',
    '\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n*New Address*\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nTel: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\ndebbiemargulies@hotmail.com\r\nwww.avocadoit.com',
    '\r\nDebbie Margulies \r\nMarketing Manager \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nTel: 408.562.8059 \r\nMobile: 408.393.1052 \r\nFax: 408.562.8100 \r\ndebbie.margulies@avocadoit.com \r\nwww.avocadoit.com \r\nYour Business Everywhere!',
    '\r\nCraig Stewart\r\nVice President\r\nGlobal Telecom Business Unit\r\nOffice: 408-562-8060\r\nFax: 408-562-8100\r\nwww.avocadoit.com',
    '\r\nSent wirelessly from AvocadoIT, Inc. \r\nPrakash Iyer\r\nFounder, CTO & Sr. Vice President, Engineering\r\nDirect: 408.562.8030 \r\npiyer@avocadoit.com \r\nAvocadoIT, Your Business Everywhere!',
    '\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nTel: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!',
    '\r\nElba Linscott\r\nSales Engineer\r\n408.562.8027 Direct\r\n408.891.8134 Cell \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nelba.linscott@avocadoit.com <mail',
    '\r\nSr. Instructor\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7956\r\nMobile: 408.857.8744\r\nFax: 408.562.8100 \r\ndoug.field@avocadoit.com\r\nPalm:  ep036@palm.net \r\nwww.avocadoit.com',
    '\r\n--------------------------\r\nSent wirelessly from AvocadoIT, Inc. \r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nTel: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com\r\nYour Business Everywhere!',
    '\r\nRon\r\nAvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nRon Silverton\r\nDirector, AvocadoITOnline\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8039\r\nmobile (415) 845-6789\r\nfax (408) 562-8100\r\nwww.avocadoit.com\r\n**********************************************',
    '\r\n--------------------------------------\r\nJimmy Kim\r\nSales Engineer\r\nAvocadoIT, Inc.\r\n4933 West Louise\r\nSuite 101\r\nSkokie, IL 60077\r\nOffice: 312-409-4919\r\nMobile: 312-405-3836\r\nFax: 810.279.4299\r\njimmy.kim@avocadoit.com\r\nwww.avocadoit.com',
    '\r\nTrish Lady\r\nVP, Business Development \r\nAvocadoIT, Inc.\r\n2211 N. 1st Street, San Jose, CA 95131\r\np:  408-562-8070  m:  415-505-3756   f:  408-562-8100  \r\nemail:  trish.lady@avocadoit.com\r\nAre you ready for the wireless web?',
    '\r\nDebbie Margulies \r\nMarketing Manager \r\nAvocadoIT, Inc. \r\n*New Address* \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nTel: 408.562.8059 \r\nMobile: 408.393.1052 \r\nFax: 408.562.8100 \r\ndebbie.margulies@avocadoit.com \r\ndebbiemargulies@hotmail.com \r\nwww.avocadoit.com',
    '\r\nVice President\r\nGlobal Telecom Business \r\n____________________________\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, California 95131\r\nMobile:   408-891-8139\r\nOffice:    408-562-8060  \r\nFax:       408-562-8100\r\nEmail:    cstewart@avocadoit.com\r\nWeb:      www.avocadoit.com',
    '---\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Sales Engineering\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w',
    '\r\n---\r\nDan Baca \t\t\tdbaca@avocadoit.com\t\t408.562.8041\r\nDirector of Engineering\t\twww.avocadoit.com\t\t408.394.6114 m',
    '\r\nElisabeth V. Whaley\r\nAssit. to the VP of Prof. Srvcs.\r\noffice: (408) 562-8189\r\nfax: (408) 562-8103\r\nElisabeth.Whaley@avocadoit.com\r\nAvocadoIT, Your Business Everywhere!',
    '\r\n----------------------------------\r\nBrad Seipp                \r\nSales Engineer          \r\n310.621.8201 W\r\nwww.avocadoit.com \r\nSent from my AvocadoIT \r\nWireless Handheld',
    '\r\n--------------------------\r\nSent To You Wirelessly From AvocadoIT (www.avocadoit.com)\r\nDavid Marcus\r\nDirector of Sales\r\nAvocadoIT, Inc .\r\n2211 North First Street\r\nSan Jose, CA  95131\r\nmobile (650) 281-9010\r\ndirect (408) 562-7978\r\ndavid.marcus@avocadoit.com'
    '\r\n________________________________________________\r\nRishi Manocha\r\nProject Manager\r\nEmail : rishi.manocha@avocadoit.com\r\nDirect: 408-562-8112\r\nMobile: 408-857-8746',
    '\r\n*******************************************************************\r\nChristopher Quinter\r\nDirector of Sales\r\n431 Trumbull Court\r\nNewtown, PA  18940\r\n(267) 757-0736  Mail\r\n(215) 778-3235  Mobile\r\n(413) 714-7699  Fax\r\nAvocadoIT, Your Business Everywhere!  www.avocadoit.com \r\n*******************************************************************\r\n',
    '\r\n--------------------------\r\nSent To You Wirelessly From AvocadoIT (www.avocadoit.com)\r\nDavid Marcus\r\nDirector of Sales\r\nAvocadoIT, Inc .\r\n2211 North First Street\r\nSan Jose, CA  95131\r\nmobile (650) 281-9010\r\ndirect (408) 562-7978\r\ndavid.marcus@avocadoit.com',
    '\r\n________________________________________________\r\nRishi Manocha\r\nProject Manager\r\nEmail : rishi.manocha@avocadoit.com\r\nDirect: 408-562-8112\r\nMobile: 408-857-8746',
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.*\r\n3965 Freedom Circle\r\nSuite 1100\r\nSanta Clara, CA 95054\r\n*We've moved (as of July 1, 2003)\r\nMain: 408.562.8000\r\nDirect: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    '\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nTel: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com\r\nYour Business Everywhere!',
    "\r\nYour Business Everywhere\r\n-- \r\nAvocadoIT Europe GmbH \r\nKönigsallee 106 \r\n40215 Düsseldorf, Germany \r\n(p)     +49 (0) 211.30122-125 \r\n(f)     +49 (0) 211.30122-200 \r\n(m)     +49 (0) 171.3845255 \r\nmailwww.avocadoit.com",
    "\r\nBrett W. Welborn\r\nAccount Executive\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)",
    "\r\nLaura C. Henderson\r\nClient Development\r\nAvocadoIT, Inc.\r\nemail: laura.henderson@avocadoit.com\r\n(800) 355.1103 ext 8187\r\nDirect: (408) 562.8187\r\nCell: (408) 857.8529\r\nhttp://www.avocadoit.com",
    "\r\nChristopher Quinter\r\nAccount Manager\r\n431 Trumbull Court\r\nNewtown, PA  18940\r\nOffice:  (267) 757-0736\r\nMobilizing solutions to reduce costs, increase revenue & improve productivity www.avocadoit.com\r\n",
    "\r\nSenior Applications Engineer\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000 \r\nDirect: 408.562.8001\r\nMobile: 408.394.3613\r\nFax: 408.562.8100\r\ndfox@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nYour Business  Everywhere\r\n***********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc\r\n2211 N First Street, Suite 200\r\nSan Jose CA, 95131\r\nw 408.562-8039\r\nm 415.845-6789\r\nf 408.5622-8100\r\nwww.avocadoit.com\r\n*************************************************",
    "\r\n---\r\nRicardo Garcia\r\nTechnical Marketing Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-7984 office\r\n(408) 562-8100 fax\r\n(408) 829-4827 cell\r\nricardo.garcia@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nAmy Moore\r\nExecutive Assistant\r\nMark Tapling, President & CEO\r\nMatt DiMaria, Sr. VP Marketing & Business Development\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8106  fax\r\n(408) 857-8520  mobile\r\nwww.avocadoit.com <http://www.avocadoit.com>\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nDavid Marcus\r\nSenior Corporate Counsel\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\ndirect (408) 562-7978\r\nmobile (650) 281-9010\r\nfax (408) 562-8100\r\ndavid.marcus@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nBrett W. Welborn\r\nAccount Development\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)",
    "\r\nKhou Fang\r\nTechnical Recruiting Manager\r\nkfang@avocadoit.com\r\n408-562-8096\r\n",
    '\r\nJohn McCann, District Manager\r\nAvocadoIT, Inc\r\n8745 W. Higgins, Suite 360\r\nChicago, IL 60631\r\nOffice:  773.243-0324\r\nMobile: 847-826-8064\r\nFax: 773-243-0225\r\nJohn.McCann@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!\r\n--------------------------\r\nSent from my Wireless Handheld',
    "\r\nRitesh Adval\r\nAvocadoIT, Inc.\r\n2211 North First Street, \r\nSuite 200, San Jose,\r\nCA  95131\r\nDirect: 408.562.8035 \r\nFax: 408.562.8100 \r\nradval@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nRon\r\nAvocadoIT, Your Business Everywhere! \r\n********************************************** \r\nRon Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc. \r\n2211 N. First Street, Suite 200 \r\nSan Jose, CA 95131 \r\nwork (408) 562-8039 \r\nmobile (415) 845-6789 \r\nfax (408) 562-8100 \r\nwww.avocadoit.com \r\n**********************************************",
    "\r\nEdward Tian\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000 \r\nDirect: 408.562.8122\r\nFax: 408.562.8100 \r\netian@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nRon\r\nAvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8039\r\nmobile (415) 845-6789\r\nfax (408) 562-8100\r\nwww.avocadoit.com\r\n**********************************************",
    "Toshiya Otani\r\nVice President, Business Development - Asia Pacific\r\nAvocadoIT, Inc.\r\nRepresentative Director\r\nAvocadoIT Japan KK\r\ntel: 650-740-9792 (cell/US), +81-90-6019-0079 (cell/Jpn)\r\ne-mail: toshiya.otani@avocadoit.com\r\nUS: 2211 North First Street Suite 200, San Jose, CA 95131\r\nJapan: 105-0001 4-3-20 Toranomon Minato-ku, Tokyo\r\nKamiyacho Mori Building 14F\r\nwww.avocadoit.com\r\n",
    "\r\nTy\r\n(415) 336-8668 Mobile\r\nAvocadoIT, Your Business Everywhere!\r\n*********************************************\r\nT. Ty Wang\r\nBusiness Development\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-7945 Office\r\n(408) 562-8100 Fax\r\nty.wang@avocadoit.com\r\n*********************************************",
    "\r\n--\r\nAvocadoIT Europe GmbH\r\nHamborner Strasse 53\r\n40472 Duesseldorf\r\nphone\t+49 (211) 94098-119\r\nfax\t+49 (211) 94098-200\r\nmobile\t+49 (171) 4014644\r\nmailwww.avocadoit.com",
    "\r\nRon (www.avocadoit.com)",
    "\r\n--------------------------\r\nRicardo Garcia                         rgarcia@avocadoit.com\r\nSales Engineer                         408-829-4827 m\r\nAvocadoIT Inc.                          408-562-7984 w\r\n---\r\nSent from my AvocadoIT mobile device (www.avocadoit.com)",
    "\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Product Marketing\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w",
    "\r\nKhou Fang\r\nTechnical Recruiting Manager\r\nkfang@avocadoit.com\r\n408-562-8096",
    "\r\n---\r\nD'Arcy Salzmann, Product Manager\r\ndarcy.salzmann@avocadoit.com \r\nAvocadoIT, Inc.\r\n49 Ontario Street, 7th floor, Box 24\r\nToronto, Ontario, M5V 2V1\r\nCanada \r\nphone    +1.416.643.3593 Toronto, Ont.\r\nphone    +1.408.562.8172 San Jose, CA. \r\nAvocadoIT, your business everywhere. \r\nClick to add my contact info to your Lotus Notes/Netscape/ACT/Outlook address book\r\n< http://my.infotriever.com/darcysalzmann>",
    "\r\nAvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8039\r\nmobile (415) 845-6789\r\nfax (408) 562-8100\r\nwww.avocadoit.com\r\n**********************************************",
    "\r\nMiyuki\r\nMiyuki Goldman\r\nQA Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7979\r\nFax: 408.562.8100\r\nmiyuki.goldman@avocadoit.com\r\nwww.avocadoit.com <http://www.avocadoit.com>",
    "\r\n----------------------------------------------------------------------------------------\r\nAmit Sethi                                            asethi@avocadoit.com\r\nSr. Director, Business Development        408.562.7966 Office\r\nAvocadoIT, Inc.                                      408.562.8100 Fax\r\n3231 Scott Blvd.                                    www.avocadoit.com\r\nSanta Clara, CA 95054",
    "\r\nBrett Welborn\r\nAccount Development Manager\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(800) 355-1103 x8186\r\n(408) 562-8186 (direct)\r\n(408) 857-8524 (mobile)\r\nAvocadoIT, Your Business Everywhere!",
    "\r\n--------------------------\r\nDan Baca                                 dbaca@avocadoit.com\r\nDirector of Sales Engr              408-394-6114 m\r\nAvocadoIT Inc.                         408-562-8041 w\r\n---\r\nAvocadoIT, Your Business Everywhere!  www.avocadoit.com",
    "\r\nKelsey Kerr\r\nDirector Sales\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n408.562.8036 (office)\r\n408.394.1054 (cell)\r\n408.562.8100 (fax)\r\nkkerr@avocadoit.com\r\nwww.avocadoit.com\r\n",
    "\r\ndbm",
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000\r\nDirect: 408.562.8059\r\nMobile: 650.766.6094\r\nFax: 408.562.8100\r\ndmargulies@avocadoit.com\r\nwww.avocadoit.com",
    "\r\n---\r\nDan Baca                                  dbaca@avocadoit.com\r\nDirector of Engineering               408.588.4057\r\nAvocadoIT                                 www.avocadoit.com",
    "\r\nRuth Gantly\r\nManager, Information Products\r\nAvocadoIT\r\n3231 Scott Blvd.\r\nSanta Clara, CA 95054\r\nemail rgantly@avocadoit.com\r\nPhone: 408.562.8033\r\nFax: 408.562.8100",
    "\r\nBest regards,\r\n-------------------------------------------------------------------------------------------------------------\r\nRichard Evans\r\nDeveloper Support Engineer\r\nL&H Speech and Language Products\r\nrevans@lhsl.com\r\nsupport@lhsl.com",
    "\r\n____________________________________\r\nDiana Y. Chen\r\nApplications Engineer\r\nOffice: (408) 562-8143\r\nCell:  (408) 829-4689\r\nDiana.Chen@avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    '\r\nBrett W. Welborn\r\nAccount Development Representative\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)',
    '\r\nBrett Welborn\r\nAccount Executive\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-8186 (direct)\r\n(408) 857-8524 (mobile)',
    '\r\nBrett W. Welborn\r\nAccount Development\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)',
    "\r\nBrett Welborn\r\nAccount Development Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-8186 (direct)\r\n(408) 857-8524 (mobile)",
    "\r\nBrett W. Welborn\r\nAccount Development Representative\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)",
    "\r\nBrett Welborn\r\nAccount Executive\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95134\r\n(800) 355-1103 x8186\r\n(408) 562-8186 (direct)\r\n(408) 857-8524 (mobile)",
    "\r\nBrett Welborn\r\nAccount Executive\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95134\r\n(800) 355-1103 x8186\r\n(408) 562-8186 (direct)\r\n(408) 857-8524 (mobile)\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nBrett W. Welborn \r\nAccount Development \r\nAvocadoIT, Inc. \r\nhttp://www.avocadoit.com \r\n2211 North First Street, Suite 200 \r\nSan Jose, CA  95131 \r\n(800) 355-1103 ext. 8186 \r\n(408) 562-8186 (direct)",
    "\r\nBrett W. Welborn\r\nAccount Development Representative\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(408) 562-8186",
    "Brett W. Welborn\r\nAccount Development Representative\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)",
    "\r\nBrett W. Welborn\r\nAccount Development Representative\r\nAvocadoIT, Inc.\r\n> 3231 Scott Boulevard\r\n> Santa Clara, CA  95054\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)",
    "\r\nElisabeth V. Whaley\r\nAssist. to the VP of Prof. Srvcs.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice: (408) 562-8189\r\nfax: (408) 562-8103\r\nElisabeth.Whaley@avocadoit.com",
    "\r\nElisabeth V. Whaley\r\nAssit. to the VP of Client Srvcs.\r\noffice: (408) 562-8189\r\nfax: (408) 562-8103\r\nElisabeth.Whaley@avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nElisabeth V. Whaley\r\nAdmin. Assist. in Prof. Srvcs.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "\r\nElisabeth V. Whaley\r\nAdmin. Assist. in Sales\r\n3231 Scott Blvd.\r\nSanta Clara, CA  95054\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "\r\nElisabeth V. Whaley\r\nAssit. to the VP of Customer Service\r\noffice: (408) 562-8189\r\nfax: (408) 562-8103\r\nElisabeth.Whaley@avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "Elisabeth V. Whaley\r\nAssit. to the VP of Customer Service\r\noffice: (408) 562-8189\r\nfax: (408) 562-8103\r\nElisabeth.Whaley@avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nElisabeth V. Whaley\r\nAdmin. Assist. in Prof. Srvcs.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95113\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "Elisabeth V. Whaley\r\nAdmin. Assist. in Prof. Srvcs.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95113\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "\r\nElisabeth V. Whaley\r\nAdmin. Assist. in Sales\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "\r\nElisabeth V. Whaley\r\nAdmin. Assist. Prof. Serv./Sales\r\n2211 North First Street\r\nSan Jose, CA  95131\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "Elisabeth V. Whaley\r\nAdmin. Assist. in Sales\r\n2211 North First Street\r\nSan Jose, CA  95131\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "Elisabeth V. Whaley\r\nAdmin. Assist. in Prof. Srvcs.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "Elisabeth V. Whaley\r\nAssist. to the VP of Prof. Srvcs.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice: (408) 562-8189\r\nfax: (408) 562-8103\r\nElisabeth.Whaley@avocadoit.com",
    "\r\nElisabeth V. Whaley\r\nAdmin. Asst. to VP of Cust. Serv.\r\n2211 North First St., Ste. 200\r\nSan Jose, CA  95131\r\nphone: 408-562-8189\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nElisabeth V. Whaley\r\nExec. Assit. to the VP of Client Srvcs.\r\noffice: (408) 562-8189\r\nfax: (408) 562-8103\r\nElisabeth.Whaley@avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nNoemi Petrone \r\nExecutive Administrator to Venk Shukla \r\nAvocadoIT \r\n3231 Scott Blvd. \r\nSanta Clara, CA \r\n408-562-8032 (direct) \r\n408-588-1519 (fax)",
    "Noemi Petrone\r\nExecutive Administrator to Venk Shukla\r\nAvocadoIT\r\n3231 Scott Blvd.\r\nSanta Clara, CA\r\n408-562-8032 (direct)\r\n408-588-1519 (fax)",
    "\r\nNoemi Petrone\r\nExecutive Administrator to Venk Shukla\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd.\r\nSanta Clara, CA\r\n408-562-8032 (direct)\r\n408-588-1519 (fax)",
    "\r\nNoemi Petrone\r\n2179 Coolidge Dr.\r\nSanta Clara, CA\r\n408-246-8811\r\n",
    "\r\nTrish Lady\r\nVP, Business Development - Content\r\nAvocadoIT\r\n3231 Scott Blvd., Santa Clara, CA 95054\r\np:  408-562-8070   f:  408-562-8100   m:  415-505-3756\r\nAre you ready for the wireless web?",
    "\r\nRegards,\r\nTrish",
    '\r\nTrish',
    "r\nhttp://concepts.bigcharts.com/marketwatch/mktw-wireless-browser/index.html \r\nJoe Techar\r\nDirector\r\nCBS MarketWatch.com\r\n612.752.1138\r\njtechar@marketwatch.com\r\n123 North 3rd St., Suite 300\r\nMinneapolis, MN 55401\r\n",
    "\r\nMike Richman\r\nSenior Account Manager\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice (forwards to cell phone): 415-460-0644\r\nMike.Richman@AvocadoIT.com\r\n*** AvocadoIT, Your Business Everywhere! ***",
    '\r\nMike\r\n--------------------------\r\nMike Richman\r\nSenior Account Executive\r\n415-460-0644 W\r\n"AvocadoIT, Your Business Everywhere!"',
    "\r\nMike Richman\r\nSenior Account Manager\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice (forwards to cell phone): 415-460-0644\r\nMike.Richman@AvocadoIT.com",
    'Mike\r\n--------------------------\r\nMike Richman\r\nSenior Account Executive\r\n415-460-0644 W\r\n"AvocadoIT, Your Business Everywhere!"',
    "\r\nMike Richman\r\nSenior Account Executive\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice (forwards to cell phone): 415-460-0644\r\nMike.Richman@AvocadoIT.com",
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nTel: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nTamra Van Steenburgh\r\nDirector of Strategic Finance\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\nMain:  408-562-8000\r\nDirect:  408-562-8087\r\nFax:  408-562-8100\r\ntamra.vansteenburgh@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nMike Richman\r\nSenior Account Manager\r\nAvocadoIT, Inc.\r\nDirect: 415.460.0644 \r\nMike.Richman@AvocadoIT.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "\r\n--> Andy\r\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector of Business Development\t\t408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8102 fax\r\nwww.avocadoit.com\t\t\t\t2211 N. 1st Street, Suite 200, San Jose, CA 95131",
    "\r\n--> Andy\r\n_____________\r\nAndrew Wong\r\nAvocadoIT, Your Business Everywhere!\r\n408.562.8131\r\nandy.wong@avocadoit.com",
    "\r\nAndy\r\n---\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector of Business Development\t\t408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8100 fax\r\nwww.avocadoit.com\t\t\t\t3231 Scott Blvd, Santa Clara, CA 95054",
    "\r\nThanks!\r\n--> Andy\r\n_____________\r\nAndrew Wong\r\nAvocadoIT - Your Business Everywhere\r\n408.562.8131\r\nandy.wong@avocadoit.com",
    "\r\nandy\r\n---\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector of Business Development\t\t408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8100 fax\r\nwww.avocadoit.com\t\t\t\t3231 Scott Blvd, Santa Clara, CA 95054",
    "\r\nThanks!\r\n--> Andy\r\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector of Business Development\t\t408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8100 fax\r\nwww.avocadoit.com\t\t\t\t3231 Scott Blvd, Santa Clara, CA 95054",
    "\r\nThanks!\r\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector of Business Development\t\t408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8100 fax\r\nwww.avocadoit.com\t\t\t\t3231 Scott Blvd, Santa\r\nClara, CA 95054",
    "\r\n--> Andy\r\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector Solutions Marketing\r\n408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8102 fax\r\nwww.avocadoit.com\t\t\t\t2211 N. 1st Street, Suite 200, San Jose, CA 95131\r\nYour Business Everywhere",
    "\r\nthanks!\r\nandy\r\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector of Business Development\t\t408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8100 fax\r\nwww.avocadoit.com\t\t\t\t3231 Scott Blvd, Santa Clara, CA 95054",
    '\r\nPeter Smialek\r\nDirector, North American Sales\r\nAvocadoIT\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile\r\n"AvocadoIT...Your Business Everywhere"',
    '\r\nPeter Smialek\r\nRegional Sales Director\r\nAvocadoIT\r\n781 238 6774',
    '\r\nPeter Smialek\r\nSales Director\r\nAvocadoIT, Inc.\r\n25 Mall Road - Suite 300\r\nBurlington, MA 01803\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile\r\n"AvocadoIT...Your Business Everywhere"',
    '\r\nPeter Smialek \r\nRegional Sales Director \r\nAvocadoIT \r\n781 238 6774',
    'Peter Smialek\r\nSales Director\r\nAvocadoIT, Inc.\r\n25 Mall Road - Suite 300\r\nBurlington, MA 01803\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile\r\n"AvocadoIT...Your Business Everywhere"',
    '\r\nPeter Smialek\r\nDirector, North American Sales\r\nAvocadoIT\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile',
    '\r\nThanks\r\nNadir Nawabzada\r\nSystem Administrator\r\nAvocadoIT, Inc.\r\n2211 North First Street Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408-562-8044\r\nCell: 408-205-7289',
    '\r\nPeter Smialek\r\nDirector\r\nAvocadoIT, Inc.\r\n25 Mall Road - Suite 300\r\nBurlington, MA 01803\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile\r\n"AvocadoIT...Your Business Everywhere"',
    "\r\n--------------------------\r\nThe message is brief because it is sent from my Blackberry\r\nVenktesh Shukla\r\nAvocadoIT : Your business everywhere",
    '\r\nPeter Smialek \r\nDirector, North American Sales \r\nAvocadoIT \r\n781 238 6774  Office \r\n781 229 0330  Fax \r\n617 901 9111  Mobile \r\n"AvocadoIT...Your Business Everywhere"',
    "\r\n[This message was sent from Blackberry handheld.]",
    "\r\nhttp://iwww.sitraka.com/mobility/rd/aa/index.html\r\nkgr\r\n--\r\nKeith Ritchie\r\nTeam Lead\r\nMobile Infrastructure\r\nSitraka Mobility\r\nhttp://www.sitraka.com/mobility\r\nWork  - (416) 366-6425 x 306\r\nCell  - (905) 467-1181\r\nFax  - (416) 214-1235\r\nEmail  - keith.ritchie@sitraka.com\r\nBlackberry - sitraka2@mobile.rogers.com\r\n",
    '\r\nLisa Zissen \r\nExecutive Administrator\r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite #200\r\nSan Jose, California  95131 \r\n(408) 562-8012 - direct \r\n(408) 205-0665 - Mobile \r\n(408) 562-8100 - Fax \r\nlisa.zissen@avocadoit.com \r\nwww.avocadoit.com \r\nAvocadoIT, your business everywhere! <?xml:namespace prefix = o ns = "urn:schemas-microsoft-com:office:office" />',
    "\r\nhttp://iwww.sitraka.com/mobility/rd/aa/may15-21.html\r\nkgr\r\n--\r\nKeith Ritchie\r\nDevelopment Manager, Respond Platform\r\nSitraka Mobility\r\nhttp://www.sitraka.com/mobility\r\nWork  - (416) 366-6425 x 306\r\nCell  - (905) 467-1181\r\nFax  - (416) 214-1235\r\nEmail  - keith.ritchie@sitraka.com\r\nBlackberry - sitraka2@mobile.rogers.com\r\n",
     "\r\nEnjoy,\r\nkgr\r\n--\r\nKeith Ritchie\r\nDevelopment Manager, Respond Platform\r\nSitraka Mobility\r\nhttp://www.sitraka.com/mobility\r\nWork  - (416) 366-6425 x 306\r\nCell  - (905) 467-1181\r\nFax  - (416) 214-1235\r\nEmail  - keith.ritchie@sitraka.com\r\nBlackberry - sitraka2@mobile.rogers.com\r\n",
    "Ritchie\r\nTeam Lead\r\nMobile Infrastructure\r\nSitraka Mobility\r\nhttp://www.sitraka.com/mobility\r\nWork  - (416) 366-6425 x 306\r\nCell  - (905) 467-1181\r\nFax  - (416) 214-1235\r\nEmail  - keith.ritchie@sitraka.com\r\nBlackberry - sitraka2@mobile.rogers.com\r\n",
    "\r\n---\r\nDan Baca\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Marketing\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t408.562.8041 w",
    'r\n--\r\nKeith Ritchie\r\nDevelopment Manager, Respond Platform\r\nSitraka Mobility\r\nhttp://www.sitraka.com/mobility\r\nWork  - (416) 366-6425 x 306\r\nCell  - (905) 467-1181\r\nFax  - (416) 214-1235\r\nEmail  - keith.ritchie@sitraka.com\r\nBlackberry - sitraka2@mobile.rogers.com\r\n',
    "\r\n--\r\nKeith Ritchie\r\nDevelopment Manager, Respond Platform\r\nSitraka Mobility\r\nhttp://www.sitraka.com/mobility\r\nWork  - (416) 366-6425 x 306\r\nCell  - (905) 467-1181\r\nFax  - (416) 214-1235\r\nEmail  - keith.ritchie@sitraka.com\r\nBlackberry - sitraka2@mobile.rogers.com\r\n",
    "\r\nkgr",
    "\r\nKeith\r\n__________________________________________________\r\nKeith Ritchie\r\nDevelopment Manager\r\nAvocadoIT Canada\r\n49 Ontario Street\r\nBox 24, 6th Floor\r\nToronto ON M5A 2V1\r\nPhone: 416.366.6425 x 306\r\nFax: 416.643.4832\r\nkeith.ritchie@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "\r\n__________________________________________________\r\nKeith Ritchie\r\nDevelopment Manager\r\nAvocadoIT Canada\r\n49 Ontario Street\r\nBox 24, 6th Floor\r\nToronto ON M5A 2V1\r\nPhone: 416.366.6425 x 306\r\nFax: 416.643.4832\r\nkeith.ritchie@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nthx,\r\nKeith Ritchie\r\nDevelopment Manager, Respond Platform\r\nSitraka Mobility\r\nhttp://www.sitraka.com/mobility\r\nWork  - (416) 366-6425 x 306\r\nCell  - (905) 467-1181\r\nFax  - (416) 214-1235\r\nEmail  - keith.ritchie@sitraka.com\r\nBlackberry - sitraka2@mobile.rogers.com\r\n",
    "\r\n--------------------------\r\nGrant Frederiksen\r\nAvocadoIT, Inc.\r\nSales Director - SE\r\n770.753.9837\r\ngrantf@avocadoit.com\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)",
    "\r\n--------------------------\r\nGrant Frederiksen\r\nAvocadoIT, Inc.\r\nSales Director - SE\r\n770.753.9837\r\ngrantf@avocadoit.com\r\nSent from my AvocadoIT Powered Wireless Handheld (www.BlackBerry.net)",
    "\r\nGrant\r\nGrant Frederiksen\r\nSales Director - Southeast\r\nAvocadoIT\r\n770.753.9837  (o)\r\n770.753.9581  (f)\r\n678.522.8238  (m)\r\ngrantf@avocadoit.com\r\nwww.avocadoit.com\r\nAre you ready for the wireless web?",
    "\r\nGrant  \r\nGrant Frederiksen\r\nSales Director - Southeast\r\nAvocadoIT\r\n770.753.9837  (o)\r\n770.753.9581  (f)\r\n678.522.8238  (m)\r\ngrantf@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nGrant Frederiksen\r\nSales Director - Southeast\r\nAvocadoIT\r\n770.753.9837  (o)\r\n770.753.9581  (f)\r\n678.522.8238  (m)\r\ngrantf@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks,\r\nGrant \r\nAre you ready for the wireless web?\r\n",
    "\r\n--------------------------------\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)",
    "\r\n--------------------------\r\nSent from my BlackBerry Wireless Handheld",
    "\r\n--------------------------\r\nLarry Salerno\r\nAvocadoIT, Inc\r\nSent from my BlackBerry Wireless Handheld",
    "\r\nLisa Zissen\r\nExecutive Assistant\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)",
    "--------------------------------\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)",
    "\r\n--------------------------\r\nR. Scott Isgitt\r\nSales Director\r\nAvocadoIT\r\n512-514-6424 - Office\r\n512-636-1275\r\nscott.isgitt@avocadoit.com\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)"
    "\r\n--------------------------\r\nStephanie Moss\r\nSent from my BlackBerry Wireless Handheld\r\n",
    "\r\nLisa Zissen\r\nExecutive Assistant\r\nAvocadoIT, Inc.\r\nSent wirelessly from my Blackberry handheld device.\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)",
    "\r\n-------------------------- \r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net) \r\n",
    "\r\n--------------------------\r\nR. Scott Isgitt\r\nSales Director\r\nAvocadoIT\r\n512-514-6424 - Office\r\n512-636-1275\r\nscott.isgitt@avocadoit.com\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)"
    "\r\nJosh\r\n--------------------------------\r\nSent from my BlackBerry Wireless Handheld\r\n(www.BlackBerry.net)\r\n",
    "--------------------------\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)\r\n",
    "--------------------------\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)",
    "\r\nRoberta Wiggins, Director Wireless Mobile\r\nYankee Group\r\nGloria\r\n____________________________\r\nGloria Chon\r\nAccount Associate\r\nCandelori Communications\r\n1270 Oakmead Parkway, Suite 314\r\nSunnyvale, CA  94086\r\nTel   408.774.3414 x227\r\nFax  408.774.3419\r\ngchon@candelori.com\r\n",
    #"\r\nEnabling Infrastructure for Post-PC Era\r\n",
    'AvocadoIT, Your Business Everywhere! \r\n********************************************** \r\nRon Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc. \r\n2211 N. First Street, Suite 200 \r\nSan Jose, CA 95131 \r\nwork (408) 562-8039 \r\nmobile (415) 845-6789 \r\nfax (408) 562-8100 \r\nwww.avocadoit.com \r\n**********************************************',
    '\r\nEnabling Infrastructure for Post-PC Era \r\n************************************ \r\nRon D Silverton \r\nDirector of Strategic Marketing \r\n3231 Scott Boulevard \r\nSanta Clara, CA  95054 \r\nTel: (408) 562-8039 \r\nMobile: (415) 845-6789 \r\nFax: (408) 408.562.8100 \r\nrsilverton@avocadoit.com \r\n************************************',
    '\r\nAvocadoIT, Your Business  Everywhere\r\n***********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc\r\n2211 N First Street, Suite 200\r\nSan Jose CA, 95131\r\nw 408.562-8039\r\nm 415.845-6789\r\nf 408.562-8100\r\nwww.avocadoit.com\r\n*************************************************',
    "\r\nEnabling Infrastructure for Post-PC Era \r\n************************************ \r\nRon D Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nTel: 408.562.8039 \r\nMobile: 415.845.6789 \r\nFax:  408.562.8100 \r\nrsilverton@avocadoit.com \r\n************************************",
    "\r\nRon\r\nAvocadoIT, Your Business  Everywhere \r\n*********************************************** \r\nRon Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc \r\n2211 N First Street, Suite 200 \r\nSan Jose CA, 95131 \r\nw 408.562-8039 \r\nm 415.845-6789 \r\nf 408.562-8100 \r\nwww.avocadoit.com \r\n*************************************************",
    "\r\nEnabling Infrastructure for Post-PC Era\r\n************************************\r\nRon D Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel: 408.562.8039\r\nMobile: 415.845.6789\r\nFax:  408.562.8100 \r\nrsilverton@avocadoit.com\r\n************************************",
    "\r\nAvocadoIT, Your Business  Everywhere \r\n*********************************************** \r\nRon Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc \r\n2211 N First Street, Suite 200 \r\nSan Jose CA, 95131 \r\nw 408.562-8039 \r\nm 415.845-6789 \r\nf 408.562-8100 \r\nwww.avocadoit.com \r\n*************************************************",
    "\r\nAvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nRon Silverton\r\nDirector, AvocadoITOnline\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8039\r\nmobile (415) 845-6789\r\nfax (408) 562-8100\r\nwww.avocadoit.com\r\n**********************************************",
    "Your Business  Everywhere\r\n***********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc\r\n2211 N First Street, Suite 200\r\nSan Jose CA, 95131\r\nw 408.562-8039\r\nm 415.845-6789\r\nf 408.5622-8100\r\nwww.avocadoit.com\r\n*************************************************",
    "Ron Enabling Infrastructure for Post-PC Era \r\n************************************ \r\nRon D Silverton \r\nDirector of Strategic Marketing \r\n3231 Scott Boulevard \r\nSanta Clara, CA  95054 \r\nTel: (408) 562-8039 \r\nMobile: (415) 845-6789 \r\nFax: (408) 408.562.8100 \r\nrsilverton@avocadoit.com \r\n************************************ \r\n",
    "\r\nYour Business  Everywhere \r\n*********************************************** \r\nRon Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc \r\n2211 N First Street, Suite 200 \r\nSan Jose CA, 95131 \r\nw 408.562-8039 \r\nm 415.845-6789 \r\nf 408.562-8100 \r\nwww.avocadoit.com \r\n*************************************************",
    "Enabling Infrastructure for Post-PC Era \r\n************************************ \r\nRon D Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nTel: 408.562.8039 \r\nMobile: 415.845.6789 \r\nFax:  408.562.8100 \r\nrsilverton@avocadoit.com \r\n************************************",
    "\r\nYour Business  Everywhere \r\n*********************************************** \r\nRon Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc \r\n2211 N First Street, Suite 200 \r\nSan Jose CA, 95131 \r\nw 408.562-8039 \r\nm 415.845-6789 \r\nf 408.5622-8100 \r\nwww.avocadoit.com \r\n*************************************************",
    "AvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8039\r\nmobile (415) 845-6789\r\nfax (408) 562-8100\r\nwww.avocadoit.com\r\n**********************************************\r\n<<Ron Silverton.vcf>>\r\n",
    "Enabling Infrastructure for Post-PC Era\r\n************************************\r\nRon D Silverton\r\nDirector of Strategic Marketing\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel: (408) 562-8039\r\nMobile: (415) 845-6789\r\nFax: (408) 408.562.8100\r\nrsilverton@avocadoit.com\r\n************************************\r\n",
    "Enabling Infrastructure for Post-PC Era \r\n************************************ \r\nRon D Silverton \r\nDirector of Strategic Marketing \r\n3231 Scott Boulevard \r\nSanta Clara, CA  95054 \r\nTel: (408) 562-8039 \r\nMobile: (415) 845-6789 \r\nFax: (408) 408.562.8100 \r\nrsilverton@avocadoit.com \r\n************************************",
    "\r\nRon\r\n***********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc\r\n2211 N First Street, Suite 200\r\nSan Jose CA, 95131\r\nw 408.562-8039\r\nm 415.845-6789\r\nf 408.562-8100\r\nwww.avocadoit.com\r\n*************************************************",
    "\r\n*********************************\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n*********************************",
    "\r\nThanks,\r\n--------------------------\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n*********************************\r\nMobilized by AvocadoIT Inc.",
    '\r\nThanks,\r\n--------------------------\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n***********************************************\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nThanks, \r\n--------------------------\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n*********************************\r\nBrought to you wirelessly by AvocadoIT Inc.',
    '\r\nThanks guys,\r\n--------------------------\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n*********************************\r\nMobilized by AvocadoIT Inc.',
    '\r\n--------------------------\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n***********************************************\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\n--------------------------\r\nR. Scott Isgitt\r\nSales Director\r\nAvocadoIT\r\n512-514-6424 - Office\r\n512-636-1275\r\nscott.isgitt@avocadoit.com\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)',
    '\r\nDon\r\n--------------------------\r\n"AvocadoIT...Your Business Everywhere"',
    '\r\nScott\r\n--------------------------\r\nAvocadoIT - Your Business Everywhere!\r\nScott Weller\r\n206-323-5105 (o)\r\n206-604-6105 (m)',
    '\r\nThanks!\r\nAndy\r\n--------------------------\r\nSent to you wirelessly from AvocadoIT - Your Business Everywhere\r\nAndy Wong\r\n408.562.8131\r\nawong@avocadoit.com',
    '\r\nAmit.\r\n-------------------------------\r\nAvocadoIT - Your Business Everywhere!',
    '\r\n--------------------------\r\n"AvocadoIT...Your Business Everywhere"',
    '--------------------------\r\n"AvocadoIT...Your Business Everywhere"',
    '--------------------------\r\nAvocadoIT , Your Business Everywhere!',
    '\r\n-------------------------------\r\nAvocadoIT - Your Business Everywhere!',
    '-------------------------------\r\nAvocadoIT - Your Business Everywhere!',
    '\r\nAvocadoIT - Your Business Everywhere!\r\n------------------------------------------------------\r\nSent wirelessly from AvocadoIT!',
    '\r\nJanie\r\n(303) 679-9779\r\nAvocadoIT - Your Business Everywhere!',
    '\r\njanie\r\nAvocadoIT - Your Business Everywhere!',
    '\r\nAvocadoIT - Your Business Everywhere!',
    '\r\nPeter Smialek\r\nAvocadoIT\r\n25 Mall Road - Suite 300\r\nBurlington,  MA  01803\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile\r\n"AvocadoIT...Your Business Everywhere"',
    '\r\nPeter Smialek \r\nSales Director \r\nAvocadoIT, Inc. \r\n25 Mall Road - Suite 300 \r\nBurlington, MA 01803 \r\n781 238 6774  Office \r\n781 229 0330  Fax \r\n617 901 9111  Mobile \r\n"AvocadoIT...Your Business Everywhere"',
    '\r\n--> Andy\r\n_____________\r\nAndrew Wong\r\nAvocadoIT - Your Business Everywhere\r\n408.562.8131\r\nandy.wong@avocadoit.com',
    '\r\n------------------------------------\r\nAvocadoIT, Your Business Everywhere!',
    '\r\n--------------------------\r\nDoug Adkins\r\nAvocadoIT, Your Business Everywhere!',
    '\r\nGermana Martinez\r\nHuman Resources Coordinator,\r\nand Executive Admin \r\nAvocadoIT, Inc.\r\n2211 North First St. Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 Phone\r\n408-562-8107 Fax\r\nAvocadoIT, Your Business Everywhere!',
    '\r\nChris\r\n***************************\r\nChristopher Quinter\r\nDirector of Sales\r\n(267) 757-0736  Main\r\n(215) 778-3235  Cell\r\n(413) 714-7699  Fax\r\n***************************\r\nAvocadoIT, Your Business Everywhere!',
    "\r\n--------------------------\r\nRicardo Garcia\r\nSales Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-7984 (direct)\r\n(408) 829-4827 (mobile)\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nChris\r\n*******************************************************************\r\nChristopher Quinter\r\nDirector of Sales\r\n431 Trumbull Court\r\nNewtown, PA  18940\r\n(267) 757-0736  Mail\r\n(215) 778-3235  Mobile\r\n(413) 714-7699  Fax\r\nAvocadoIT, Your Business Everywhere!  www.avocadoit.com \r\n*******************************************************************",
    "\r\nAvocadoIT, Your Business Everywhere!\r\n*********************************************\r\nRobert Moore\r\nBusiness Development\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\nDirect: 408.562.8078\r\nMobile: 408.420.3852\r\nFax: 408.562.8100\r\nrobert.moore@avocadoit.com\r\n*********************************************",
    '\r\nAvocadoIT, Your Business Everywhere!\r\n--\r\nAvocadoIT Europe GmbH\r\nKönigsallee 106\r\n40215 Düsseldorf, Germany\r\nphone\t+49 (0) 211.30122-125\r\nfax \t+49 (0) 211.30122-200\r\ncellular\t+49 (0) 171.3845255\r\nmailwww.avocadoit.de',
    '\r\n-Howard\r\n____________________________\r\nHoward Mora\r\nQA Director\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-7954 (direct)\r\n(408) 562-8000 (main)\r\n(408) 562-8100 (fax)\r\nAvocadoIT, Your Business Everywhere!',
    '\r\nJohn McCann, District Manager\r\nAvocadoIT, Inc\r\n8745 W. Higgins, Suite 360\r\nChicago, IL 60631\r\nOffice:  773.243-0324\r\nMobile: 847-826-8064\r\nFax: 773-243-0225\r\nJohn.McCann@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!\r\n---------------------',
    '\r\nMatt\r\nAvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nMatt DiMaria\r\nSVP Business Operations\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8040\r\nwww.avocadoit.com\r\n**********************************************',
    "\r\n--------------------------\r\nA Wireless Message from AvocadoIT\r\nAvocadoIT Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nOffice:  408-562-8052\r\nMobile: 408-857-8729\r\nE-mail: Carlos.Restrepo@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    '\r\nThanks!\r\nAndy\r\n--------------------------\r\nSent to you wirelessly from AvocadoIT, Your Business Everywhere!\r\nAndy Wong\r\n408.562.8131\r\nawong@avocadoit.com',
    "\r\nGermana Martinez\r\nAdministrative Services Coordinator,\r\nAvocadoIT, Inc.\r\n2211 North First St. Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 Phone\r\n408-562-8107 Fax\r\nAvocadoIT, Your Business Everywhere!",
    '\r\n"AvocadoIT, Your Business Everywhere!" \r\nNobuyuki Takaki\r\nBusiness Development Director\r\nAvocadoIT Japan K.K.\r\nMail:nobuyuki.takaki@avocadoit.com\r\nPhone:81-3-5404-3418\r\nHP: www.avocadoit.com/jp',
    '\r\nEnjoy,\r\nJimmy\r\n--------------------------------------\r\nJimmy Kim\r\nSales Engineer\r\nAvocadoIT, Inc.\r\n4933 West Louise\r\nSuite 101\r\nSkokie, IL 60077\r\nOffice: 312-409-4919\r\nMobile: 312-405-3836\r\nFax: 810.279.4299\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!',
    "\r\nThanks,\r\nTom\r\nThomas J. Stoddard \r\nDistrict Manager, West \r\nAvocadoIT, Inc. \r\n2211 North First Street, Suite 200 \r\nSan Jose, CA 95131 \r\n949 425-9826 Direct\r\ntom.stoddard@avocadoit.com \r\nAvocadoIT, Your Business Everywhere!",
    '\r\nThanks\r\nRavi  Dadi\r\nAvocadoIT, Inc\r\n408-562-8138\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nDebbie Margulies \r\nMarketing Manager \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nMain: 408.562.8000 \r\nDirect: 408.562.8059 \r\nMobile: 408.393.1052 \r\nFax: 408.562.8100 \r\ndebbie.margulies@avocadoit.com \r\nwww.avocadoit.com \r\nAvocadoIT, Your Business Everywhere!'
    '\r\nSincerely,\r\nJohn Schemena\r\nDirector\r\nApplications Engineering\r\nAvocadoIT Inc.\r\nTel.:  408.562.8028\r\nFAX: 408.562.8100\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\njohn.schemena@avocadoit.com <mail\r\nAvocadoIT, Your Business Everywhere!',
    "--------------------------\r\nAvocadoIT, Your Business Everywhere! (www.avocadoit.com)",
    '\r\n____________________________________\r\nDiana Y. Chen\r\nApplications Engineer\r\nOffice: (408) 562-8143\r\nDiana.Chen@avocadoit.com\r\nAvocadoIT, Your Business Everywhere!',
    '\r\n***************************\r\nChristopher Quinter\r\nDirector of Sales\r\n(267) 757-0736  Main\r\n(215) 778-3235  Cell\r\n(413) 714-7699  Fax\r\n***************************\r\nAvocadoIT, Your Business Everywhere!',
    '\r\nThank you and have a great day! \r\nChris \r\n******************************************************************* \r\nChristopher Quinter \r\nDirector of Sales \r\n431 Trumbull Court \r\nNewtown, PA  18940 \r\n(267) 757-0736  Mail \r\n(215) 778-3235  Mobile \r\n(413) 714-7699  Fax \r\nAvocadoIT, Your Business Everywhere!  www.avocadoit.com \r\n*******************************************************************',
    '\r\nThank you and have a great day!\r\nChris\r\nChristopher Quinter\r\nAccount Manager\r\n431 Trumbull Court\r\nNewtown, PA  18940\r\n(267) 757-0736  Office\r\n(215) 778-3235  Mobile\r\nAvocadoIT, Your Business Everywhere! www.avocadoit.com\r\n',
    'dbm\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com \r\nAvocadoIT, Your Business Everywhere!',
    '\r\nSincerely,\r\nJohn Schemena\r\nDirector\r\nApplications Engineering\r\nAvocadoIT Inc.\r\nTel.:  408.562.8028\r\nFAX: 408.562.8100\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\njohn.schemena@avocadoit.com <mail\r\nAvocadoIT, Your Business Everywhere!',
    "\r\nDebbie Margulies \r\nMarketing Manager \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nMain: 408.562.8000 \r\nDirect: 408.562.8059 \r\nMobile: 408.393.1052 \r\nFax: 408.562.8100 \r\ndebbie.margulies@avocadoit.com \r\nwww.avocadoit.com \r\nAvocadoIT, Your Business Everywhere!",
    "\r\nRegards,\r\nAvocadoIT, Your Business Everywhere!\r\nJeff Newman",
    "\r\nOtto Mok\r\nTechnical Team Lead\r\nAvocadoIT Canada\r\nMain: 416.366.6425 x865\r\nDirect: 416.643.4865\r\nMobile: 416.346.3834\r\nFax: 416.643.4832\r\nEmail: otto.mok@avocadoit.com\r\nWebsite: http://www.avocadoit.com/\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nThank you and have a great day!\r\nChris\r\nChristopher Quinter\r\nDirector of Sales\r\n431 Trumbull Court\r\nNewtown, PA  18940\r\n(267) 757-0736  Main\r\n(215) 778-3235  Mobile\r\n(413) 714-7699  Fax\r\nAvocadoIT, Your Business Everywhere!  www.avocadoit.com",
    "\r\nTy\r\nAvocadoIT, Your Business Everywhere!\r\n*********************************************\r\nT. Ty Wang\r\nBusiness Development\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-7945 Office\r\n(408) 562-8100 Fax\r\nty.wang@avocadoit.com\r\n*********************************************",
    "\r\nTy\r\nT. Ty Wang\r\nAvocadoIT, Inc.\r\nBusiness Development\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-7945 Office   (408) 562-8100 Fax\r\nty.wang@avocadoit.com",
    "\r\nTy\r\nT. Ty Wang\r\nDirector, Business Development\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7945\r\nFax: 408.562.8100\r\nty.wang@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "\r\n-------------------\r\nAvocadoIT, Your Business Everywhere!",
    '\r\n--------------------------\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nThanks,\r\nTom\r\ntom.stoddard@avocadoit.com\r\n949 425-9826  Office\r\n949 300-3372  Mobile\r\nAvocadoIT, Your Business Everywhere!\r\nwww.avocadoit.com',
    "\r\nThanks!\r\n--> Andy\r\n_____________\r\nAndy Wong\r\nDirector, Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8131\r\nFax: 408.562.8131\r\nawong@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    '\r\nRay\r\n------------------------------\r\nRay Canuel \r\nSales Engineer          \r\n703.281.4223  W\r\n703.728.2007  M\r\n"AvocadoIT, Your Business Everywhere!"',
    "Germana Martinez\r\nAdministrative Services Coordinator,\r\nAvocadoIT, Inc.\r\n2211 North First St. Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 Phone\r\n408-562-8107 Fax\r\nAvocadoIT, Your Business Everywhere!",
    '\r\nRajeev\r\n--------------------------------------------\r\nRajeev Mohindra\r\nFounder & V.P. of Engineering\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8002\r\nFax: 408.562.8100\r\nrajeev@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!',
    'Your Business Everywhere\r\n-- \r\nAvocadoIT Europe GmbH \r\nKönigsallee 106 \r\n40215 Düsseldorf, Germany \r\n(p)     +49 (0) 171.3845255 \r\n(f)     +49 (0) 171.13.3845255\r\nmailwww.avocadoit.com',
    '\r\nAvocadoIT, Your Business Everywhere!',
    '\r\nThanks,\r\nNobuyuki Takaki\r\nBusiness Development Director\r\nAvocadoIT Japan K.K.\r\nKamiyacho Mori Bldg. 14th Floor\r\n4-3-20 Toranomon Minato-Ku\r\nTokyo 105-0001 Japan\r\nMail:nobuyuki.takaki@avocadoit.com\r\nPhone:81-3-5404-3418\r\nHP: www.avocadoit.com/jp\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nNobuyuki Takaki\r\nBusiness Development Director\r\nAvocadoIT Japan K.K.\r\nKamiyacho Mori Bldg. 14th Floor\r\n4-3-20 Toranomon Minato-Ku\r\nTokyo 105-0001 Japan\r\nMail:nobuyuki.takaki@avocadoit.com\r\nPhone:81-3-5404-3418\r\nHP: www.avocadoit.com/jp\r\n"AvocadoIT, Your Business Everywhere!"',
    '-------------------------------------------------------------------\r\n** AvocadoIT, Your Business Everywhere **',
    '\r\nJohn McCann, District Manager\r\nAvocadoIT, Inc.\r\n8745 W. Higgins, Suite 360\r\nChicago, IL 60631\r\nOffice 773-243-0324\r\nFax  773-243-0225\r\nwww.AvocadoIT.com\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nRegards,\r\nSanjay Dubey\r\nAvocadoIT Inc. R&D\r\nsanjay.dubey@avocadoit.com\r\n(408) 562 7975\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nRavi  Dadi\r\nAvocadoIT, Inc\r\n408-562-8138\r\n"AvocadoIT, Your Business Everywhere!"',
    'AvocadoIT, Your Business Everywhere!\r\n*********************************************\r\nRobert Moore\r\nBusiness Development\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\nDirect: 408.562.8078\r\nMobile: 408.420.3852\r\nFax: 408.562.8100\r\nrobert.moore@avocadoit.com\r\n*********************************************\r\n',
    '--------------------------\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nRay Canuel \r\nSales Engineer          \r\n703.281.4223  W\r\n703.728.2007  M\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nJohn McCann, District Manager\r\nAvocadoIT, Inc.\r\n8745 W. Higgins, Suite 360\r\nChicago, IL 60631\r\nOffice 773-243-0324\r\nFax  773-243-0225\r\n"AvocadoIT, Your Business Everywhere!"',
    "\r\nMark Stewart\r\nAvocadoIT, Inc.\r\nYour Business Everywhere",
    '\r\nRoopak\r\nRoopak Parikh \r\nAvocadoIT Inc. R&D \r\nroopak@avocadoit.com \r\noff: (408) 562 8023 \r\ncell: (408 )857 8531 \r\n“AvocadoIT, Your Business Everywhere!”',
    'Mike Richman\r\nSenior Account Manager\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice (forwards to cell phone): 415-460-0644\r\nMike.Richman@AvocadoIT.com\r\n*** AvocadoIT, Your Business Everywhere! ***',
    'r\nThanks.\r\nAlex\r\n--------------------------\r\nAlex Tran\r\nDirector of Product Marketing\r\n408.562.8068 Work\r\n408.420.3877 Mobile\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nDiana\r\n____________________________________\r\nDiana Y. Chen\r\nApplications Engineer\r\nOffice: (408) 562-8143\r\nDiana.Chen@avocadoit.com\r\nAvocadoIT, Your Business Everywhere.',
    '\r\nThanks.\r\nAlex\r\n--------------------------\r\nAlex Tran\r\nDirector of Product Marketing\r\n408.562.8068 Work\r\n408.420.3877 Mobile\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\n******************************************************************* \r\nChristopher Quinter \r\nDirector of Sales \r\n431 Trumbull Court \r\nNewtown, PA  18940 \r\n(267) 757-0736  Mail \r\n(215) 778-3235  Mobile \r\n(413) 714-7699  Fax   www.avocadoit.com \r\n*******************************************************************',
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT, Your Business Everywhere!",
    "AvocadoIT, Your Business Everywhere!\r\n*********************************************\r\nRobert Moore\r\nBusiness Development\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\nDirect: 408.562.8078\r\nMobile: 408.420.3852\r\nFax: 408.562.8100\r\nrobert.moore@avocadoit.com\r\n*********************************************",
    "\r\nChristopher Quinter\r\nDirector of Sales\r\n(267) 757-0726  Office\r\n(215) 778-3235  Cell\r\nAvocadoIT, Your Business Everywhere",
    '\r\nRoopak\r\nRoopak Parikh\r\nAvocadoIT Inc. R&D\r\nroopak@avocadoit.com\r\noff: (408) 562 8023\r\ncell: (408 )857 8531\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\n"AvocadoIT, Your Business Everywhere!"\r\nNobuyuki Takaki\r\nBusiness Development Director\r\nAvocadoIT Japan K.K.\r\nMail:nobuyuki.takaki@avocadoit.com\r\nPhone:81-3-5404-3418\r\nHP: www.avocadoit.com/jp',
    '\r\n*** AvocadoIT, Your Business Everywhere! ***\r\n',
    'AvocadoIT, Your Business Everywhere!\r\n',
    'AvocadoIT, Your Business Everywhere! \r\n********************************************** \r\nMatt DiMaria \r\nSVP, Marketing and Business Development \r\nAvocadoIT, Inc. \r\n2211 N. First Street, Suite 200 \r\nSan Jose, CA 95131 \r\nwork (408) 562-8040 \r\nwww.avocadoit.com \r\n**********************************************',
    '--------------------------\r\n"AvocadoIT... Your Business Everywhere"',
    '**********************************************\r\nMatt DiMaria\r\nSVP Business Operations\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8040\r\nwww.avocadoit.com\r\n**********************************************',
    '**********************************************\r\nMatt DiMaria\r\nSVP, Marketing and Business Development\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8040\r\nwww.avocadoit.com\r\n**********************************************',
    '**********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8039\r\nmobile (415) 845-6789\r\nfax (408) 562-8100\r\nwww.avocadoit.com\r\n**********************************************',
    'Peter Smialek\r\nDirector, North American Sales\r\nAvocadoIT\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile\r\n"AvocadoIT...Your Business Everywhere"',
    '\r\njm\r\nJaime L. McNeely\r\neMarketing Manager\r\nAvocadoIT, Inc.\r\nYour Business Everywhere!\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8015 ~ phone\r\n408.891.8144 ~ cell\r\nwww.avocadoit.com',
    '\r\nThanks!\r\n"AvocadoIT, Your Business Everywhere!" \r\nNaruhito Morooka\r\nAvocadoIT Japan K.K.\r\nTEL: 03-5404-3420\r\nFAX: 03-5404-3401\r\nURL: http://www.avocadoit.com\r\nMail',
    '\r\nRoopak Parikh\r\nAvocadoIT Inc. R&D\r\nroopak@avocadoit.com\r\noff: (408) 562 8023\r\ncell: (408 )857 8531\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nNobu\r\nNobuyuki Takaki\r\nSenior Director\r\nBusiness Development & Marketing\r\nAvocadoIT Japan K.K.\r\nKamiyacho MT Bldg. 14th Floor\r\n4-3-20 Toranomon Minato-Ku\r\nTokyo 105-0001 Japan\r\nMail:nobuyuki.takaki@avocadoit.com\r\nPhone:81-3-5404-3418\r\nHP: http://www.avocadoit.co.jp/\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nNobuyuki Takaki\r\nSenior Director\r\nBusiness Development & Marketing\r\nAvocadoIT Japan K.K.\r\nKamiyacho MT Bldg. 14th Floor\r\n4-3-20 Toranomon Minato-Ku\r\nTokyo 105-0001 Japan\r\nMail:nobuyuki.takaki@avocadoit.com\r\nPhone:81-3-5404-3418\r\nHP: http://www.avocadoit.co.jp/\r\n"AvocadoIT, Your Business Everywhere!"',
    '"AvocadoIT, Your Business Everywhere!" \r\nNobuyuki Takaki\r\nBusiness Development Director\r\nAvocadoIT Japan K.K.\r\nMail:nobuyuki.takaki@avocadoit.com\r\nPhone:81-3-5404-3418\r\nHP: www.avocadoit.com/jp',
    '\r\nLarry\r\nLarry Salerno\r\nAvocadoIT, Inc\r\nOffice: 847-277-0600\r\nFax:    847-277-0640\r\nMobile:847-922-1006\r\n"AvocadoIT, Your Business Everywhere!"',
    'Your Business Everywhere\r\n-- \r\nAvocadoIT Europe GmbH \r\nKönigsallee 106 \r\n40215 Düsseldorf, Germany \r\n(p)     +49 (0) 211.30122-125 \r\n(f)     +49 (0) 211.30122-200 \r\n(m)     +49 (0) 171.3845255 \r\nmailwww.avocadoit.com',
    '\r\nRegards,\r\nAlex\r\n--------------------------\r\nAlex Tran\r\nDirector of Product Marketing\r\n408.562.8068 Work\r\n408.420.3877 Mobile\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nRegards,\r\nAvocadoIT, Your Business Everywhere\r\n============================================\r\nDavid M. Hosch,  Sales Manager\r\nAvocadoIT Inc.\r\n49 Ontario Street, 7th Floor\r\nToronto, Ontario,Canada, M5A  2V1\r\nTel: 416-643-4829, Cell: 416-460-3557\r\nFax:416-214-1235\r\nEmail: david.hosch@avocadoit.com\r\nBlackberry: david.hosch@mobile.rogers.com\r\n',
    '\r\n"AvocadoIT, Your Business Everywhere!" \r\nNaruhito Morooka\r\nDirector of Sales & Marketing\r\nAvocadoIT Japan K.K.\r\nTEL: 03-5404-3420\r\nFAX: 03-5404-3401\r\nURL: http://www.avocadoit.com\r\nMail',
    '\r\nGermana Martinez\r\nHuman Resources Coordinator, \r\nSales Executive Admin. and\r\nExecutive Assistant to the CFO\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX\r\nAvocadoIT, Your Business Everywhere',
    '\r\nGermana Martinez\r\nHuman Resources Coordinator, \r\nSales Executive Admin. and\r\nExecutive Assistant to the CFO\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX\r\nAvocadoIT, Your Business Everywhere',
    '\r\nGermana Martinez\r\nHuman Resources\r\nAvocadoIT, Your Business Everywhere',
    "\r\nDavid Swanson\r\nHuman Resources\r\nAvocadoIT, Your Business Everywhere",
    "\r\nRegards,\r\nAlex Tran\r\nDirector of Product Marketing\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(408) 562-8068 office\r\n(408) 562-8100 fax\r\n(408) 420-3877 cell\r\natran@avocadoit.com",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\nAvocadoIT\r\n650.270.4309\r\nwww.avocadoit.com",
    "\r\nLisa Zissen\r\nExecutive Assistant to Prakash Iyer\r\nFounder, CTO and Sr. VP of Engineering Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\n2nd Floor\r\nSan Jose, California  95131\r\n(408) 562-8012 - direct\r\n(408) 562-8104 - Fax\r\nlisa.zissen@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\n-Lisa\r\nLisa Zissen\r\nExec. Asst. to Prakash Iyer, CTO & Founder\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\n2nd Floor\r\nSan Jose, California  95131\r\n(408) 562-8012 - direct\r\n(408) 562-8104 - Fax\r\nlisa.zissen@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nLisa Zissen\r\nExecutive Assistant to Prakash Iyer\r\nFounder, CTO and Sr. VP of Engineering Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite #200\r\nSan Jose, CA  95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8012\r\nFax: 408.562.8104\r\nlisa.zissen@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nLisa Zissen \r\nExecutive Assistant to Prakash Iyer\r\nFounder, CTO and Sr. VP of Engineering Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\n2nd Floor\r\nSan Jose, California  95131\r\n(408) 562-8012 - direct\r\n(408) 205-0665 - Mobile\r\n(408) 562-8100 - Fax\r\nlisa.zissen@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nLisa\r\nLisa Zissen \r\nExecutive Assistant to Prakash Iyer \r\nFounder, CTO and Sr. VP of Engineering Operations \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\n2nd Floor \r\nSan Jose, California  95131 \r\n(408) 562-8012 - direct \r\n(408) 205-0665 - Mobile \r\n(408) 562-8100 - Fax \r\nlisa.zissen@avocadoit.com \r\nwww.avocadoit.com \r\nAvocadoIT, your business everywhere! \r\n_____  \r\nUpgrade Your Email - Click here!",
    "\r\nThanks.\r\nStuart\r\nStuart Finn\r\nSenior Vice President,\r\nSales and Service\r\nAVOCADOIT, INC.\r\n2211 N. First Street, Suite 200 \r\nSan Jose, CA 95131 \r\n408-562-7950 (Office) \r\nstuart.finn@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nTamra Van Steenburgh\r\nFinance Department\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd.\r\nSanta Clara, CA  95054\r\nMain:  408-562-8000\r\nDirect:  408-562-8087\r\nFax:  408-562-8100\r\ntvansteenburgh@avocadoit.com",
    "\r\nSusan Simmons\r\nDirector of Finance & Administration\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(408) 562-8022 direct dial\r\n(408) 562-8000 corporate\r\n(408) 562-8100 fax\r\nssimmons@avocadoit.com",
    "\r\nKelsey Kerr\r\nDirector Sales\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n408.562.8036 (office)\r\n408.394.1054 (cell)\r\n408.562.8100 (fax)\r\nkkerr@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000\r\nDirect: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndmargulies@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nAndrea Peterson\r\nSales Administrative Assistant\r\n3231 Scott Blvd. \r\nSanta Clara, CA  95054\r\np(408) 562-8056\r\nf(408) 562-8100\r\napeterson@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nRegards,\r\nPhilip O'Connell\r\nDirector of Sales\r\nAvocadoIT\r\nTelephone 212-749-0852\r\nFax 309-416-3937\r\nhttp://www.avocadoit.com",
    "\r\nRyan Glover\r\nPublic Relations Coordinator \r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n408.562.8057 office\r\n408.420.3841 mobile\r\nrglover@avocadoit.com\r\nhttp://www.avocadoit.com",
    "\r\nDavid Margulies\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8119\r\nDavid.Margulies@avocadoit.com\r\nthanks,\r\nRobert Moore",
    "\r\n--- \r\nD'Arcy Salzmann, Product Manager \r\ndarcy.salzmann@avocadoit.com \r\nAvocadoIT, Inc. \r\n260 King Street East, 3rd Floor \r\nToronto, ON, Canada M5A 4L5 \r\n+1.416.643.3593 phone / +1.416.605.1076 fax \r\nAvocadoIT, your business everywhere. \r\nClick to add my contact info to your Lotus Notes/Netscape/ACT/Outlook address book \r\nhttp://my.infotriever.com/darcysalzmann \r\n-----",
    "\r\n--- \r\nD'Arcy Salzmann, Product Manager \r\ndarcy.salzmann@avocadoit.com \r\nAvocadoIT, Inc. \r\n260 King Street East, 3rd Floor \r\nToronto, ON, Canada M5A 4L5 \r\n+1.416.643.3593 phone / +1.416.605.1076 fax \r\nAvocadoIT, your business everywhere. \r\nClick to add my contact info to your Lotus Notes/Netscape/ACT/Outlook address book \r\nhttp://my.infotriever.com/darcysalzmann",
    "\r\n--------------------------\r\nSent To You Wirelessly From AvocadoIT (www.avocadoit.com)\r\nDavid Marcus\r\nDirector of Sales\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSan Jose, CA  95131\r\nmobile (650) 281-9010\r\ndirect (408) 562-7978\r\ndavid.marcus@avocadoit.com",
    "\r\n----------------------------------------------------------------------------------------\r\nAmit Sethi                                            asethi@avocadoit.com\r\nVice-President, Business Development   408.562.7966 Office\r\nAvocadoIT, Inc.                                      408.562.8100 Fax\r\n2211 N. First Street, St. 200                   www.avocadoit.com\r\nSan Jose, CA 95131",
    "\r\nRepresentative Director\r\nAvocadoIT Japan KK\r\ntel: 650-740-9792 (cell/US), +81-90-6019-0079 (cell/Jpn)\r\ne-mail: toshiya.otani@avocadoit.com\r\nUS: 2211 North First Street Suite 200, San Jose, CA 95131\r\nJapan: 105-0001 4-3-20 Toranomon Minato-ku, Tokyo\r\nKamiyacho Mori Building 14F\r\nwww.avocadoit.com",
    "\r\nKind Regards,\r\nLaura\r\nLaura C. Henderson\r\nClient Development\r\nAvocadoIT, Inc.\r\nemail: laura.henderson@avocadoit.com\r\n(800) 355.1103  ext 8187\r\nDirect: (408) 562.8187\r\nhttp://www.avocadoit.com",
    "Lisa Quaglietti\r\nVP Client Development & Services\r\nAvocadoIT\r\n3231 Scott Blvd\r\nSanta Clara, CA 95054\r\nlisaq@avocadoit.com\r\n408-562-8120 - Direct\r\n408-420-3860 - Cell",
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel: (408) 562-8059\r\nMobile: (650) 766-6094\r\nFax: (408) 588-0060\r\ndmargulies@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nAndrew Wagoner\r\nPurchasing Agent\r\n408.562.8174 Direct \r\n3231 Scott Blvd\r\nSanta Clara, CA 95054\r\nandrew.wagoner@avocadoit.com",
    "\r\nSusan Simmons\r\nDirector of Finance & Administration\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\nDirect dial: (408) 562-8022\r\nCorporate phone: (408) 562-8000\r\nCorporate fax: (408) 562-8100\r\nsusan.simmons@avocadoit.com",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT, Inc. ",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT, the leader in mobile wireless platforms for the Corporate market\r\n-----------------------------",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT - the leading enterprise mobility company",
    "\r\nSent wirelessly from AvocadoIT - The leading Enterprise Mobility Company\r\nDavid C. Meltzer, Esq.  Director of Sales AvocadoIT",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT - The leading Enterprise Mobility Company Philip O'Connell, Esq.  Director of Sales AvocadoIT",
    "\r\n-------------------------------\r\nSent wirelessly from AvocadoIT - the leading enterprise mobility company",
    "\r\nThanks,\r\nTy\r\n--------------------------\r\nSent wirelessly by AvocadoIT from my BlackBerry Handheld",
    "--------------------------\r\nSent wirelessly from AvocadoIT, Inc. ",
    "\r\nElba\r\nSent wirelessly from AvocadoIT, Inc.",
    "\r\n----------------------------------\r\nJohn Schemena                \r\nDirector Applications Engineering          \r\n408 562-8028  W\r\n408 829-8236  M\r\nwww.avocadoit.com \r\nSent wirelessly from AvocadoIT - the leading enterprise mobility company",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT.",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT, the leader in mobile wireless platforms for the Corporate market\r\n------------------------",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT, Inc.",
    "\r\nAmit.\r\n-------------------------------------------------\r\nSent wirelessly from AvocadoIT - the leading enterprise mobility company",
    "\r\nTy\r\n--------------------------\r\nSent wirelessly by AvocadoIT from my BlackBerry Handheld",
    "\r\n-------------------\r\nSent wirelessly from AvocadoIT, Inc.",
    "\r\n--------------------------\r\nSent wirelessly by AvocadoIT from my BlackBerry Handheld",
    '\r\nThank you\r\nBrian\r\nBrian Robinson\r\nTechnical Support Engineer\r\n408 562 7952 (W)\r\n408 857 8740 (C)\r\n2211 N. 1st. Suite 200,\r\nSan Jose, Ca 95131\r\n"Sent wirelessly from AvocadoIT - the leading enterprise mobility company"',
    "\r\nJS\r\n------------------------------------------------------\r\nSent wirelessly from AvocadoIT!",
    "--------------------------\r\nSent wirelessly from AvocadoIT, the leader in mobile wireless platforms for the Corporate market\r\n-----------------------------",
    "\r\nSent wirelessly from AvocadoIT, Inc.",
    '\r\nBrian Robinson\r\nTechnical Support Engineer\r\n408 562 7952 (W)\r\n408 857 8740 (C)\r\n2211 N. 1st. Suite 200,\r\nSan Jose, Ca 95131\r\n"Sent wirelessly from AvocadoIT - the leading enterprise mobility company"',
    "\r\nJanie\r\n------------------------------------------------------\r\nSent wirelessly from AvocadoIT!",
    "Sent wirelessly from AvocadoIT, Inc. \r\nPrakash Iyer\r\nFounder, CTO & Sr. Vice President, Engineering\r\nDirect: 408.562.8030 \r\npiyer@avocadoit.com ",
    "\r\nPrakash\r\nPrakash Iyer\r\nFounder, CTO & Sr. Vice President, Engineering\r\nDirect: 408.562.8030\r\npiyer@avocadoit.com\r\n",
    "Sent wirelessly from AvocadoIT, Inc.",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT",
    "--------------------------\r\nSent wirelessly from AvocadoIT - the leading enterprise mobility company",
    "\r\n------------------------------------------------------\r\nSent wirelessly from AvocadoIT!",
    "Sent wirelessly from AvocadoIT - The leading Enterprise Mobility Company\r\nDavid C. Meltzer, Esq.  Director of Sales AvocadoIT",
    "\r\n---\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Product Marketing\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w",
    "\r\nThanx,\r\nDan.\r\n---\r\nDan Baca                                  dbaca@avocadoit.com\r\nDirector of Engineering               408.588.4057, c 408.394.6114\r\nAvocadoIT                                 www.avocadoit.com\r\n",
    "\r\nThanx,\r\nDan.\r\n---\r\nDan Baca                                  dbaca@avocadoit.com\r\nDirector of Engineering               408.562.8041, c 408.394.6114\r\nAvocadoIT                                 www.avocadoit.com",
    "\r\n--- \r\nDan Baca                                                dbaca@avocadoit.com \r\nDirector of Sales Engineering                   408.394.6114 m \r\nwww.avocadoit.com                                       408.562.8041 w",
    "\r\n---\r\nDan Baca\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Marketing\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t408.562.8041 w",
    "\r\n--------------------------\r\nDan Baca                                 dbaca@avocadoit.com\r\nDirector of Sales Engr              408-394-6114 m\r\nAvocadoIT Inc.                         408-562-8041 w\r\n---\r\nGet mobilized with AvocadoIT  (www.avocadoit.com)",
    "\r\n---\r\nDan Baca                                  dbaca@avocadoit.com\r\nDirector of Engineering               408.562.8041, c 408.394.6114\r\nAvocadoIT                                 www.avocadoit.com",
    "\r\n---\r\nDan Baca                                  dbaca@avocadoit.com\r\nDirector of Engineering               408.588.4057, c 408.394.6114\r\nAvocadoIT                                 www.avocadoit.com",
    "---\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Sales Engineering\t\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w",
    "--- \r\nDan Baca                                                dbaca@avocadoit.com \r\nDirector of Sales Engineering                   408.394.6114 m \r\nwww.avocadoit.com                                       408.562.8041 w",
    "\r\nDan Baca \t\t\tdbaca@avocadoit.com\t\t408.562.8041\r\nDirector of Engineering\t\twww.avocadoit.com\t\t408.394.6114 m",
    "Dan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Sales Engineering \t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w",
    "\r\n--- \r\nDan Baca                                              dbaca@avocadoit.com \r\nDirector of Technical Marketing               408.394.6114 m \r\nwww.avocadoit.com                               408.562.8041 w \r\n",
    "\r\n--------------------------\r\nDan Baca                                 dbaca@avocadoit.com\r\nDirector of Sales Engr              408-394-6114 m\r\nAvocadoIT Inc.                         408-562-8041 w\r\n---\r\nSent from my AvocadoIT mobile device (www.avocadoit.com)",
    "\r\n---\r\nRicardo Garcia                   \t\t\tricardo.garcia@avocadoit.com\r\nTechnical Marketing Engineer            \t408.829.4827 mobile\r\nwww.avocadoit.com                           \t408.562.7984 work",
    "\r\nCraig Stewart\r\nVice President\r\nGlobal Telecom Business Unit\r\n________________________________\r\nAvocadoIT\r\n3231 Scott Boulevard\r\nSanta Clara, California 95054\r\nOffice:    408-562-8060\r\nFax:       408-562-8100\r\nMobile:   510-409-7173\r\nWeb:      www.avocadoit.com",
    "---\r\nDan Baca                                  dbaca@avocadoit.com\r\nDirector of Engineering               408.588.4057\r\nAvocadoIT                                 www.avocadoit.com",
    "--------------------------\r\nDan Baca                                 dbaca@avocadoit.com\r\nDirector of Sales Engr              408-394-6114 m\r\nAvocadoIT Inc.                         408-562-8041 w\r\n---  www.avocadoit.com",
    "Dan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Product Marketing\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w",
    "\r\n--- \r\nDan Baca                                              dbaca@avocadoit.com \r\nDirector of Technical Marketing               408.394.6114 m \r\nwww.avocadoit.com                               408.562.8041 w",
    "Dan Baca\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Marketing\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t408.562.8041 w",
    "---\r\nDan Baca\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Marketing\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t408.562.8041 w",
    "\r\n_______________________________\r\nReporting Division\r\nCustomer Service Dept.\r\nWebEx Communications, Inc.\r\nReports@webex.com\r\n1.408.435.7088 option #5 \r\nwww.webex.com\r\nBringing the meeting to you.\r\n",
    "\r\nEric Shurtleff\r\nCorporate Account Manager\r\nWebEx Communications, Inc.\r\n* 408.435.7253\r\nTech Support:          support@webex.com;   408-435-7088\r\nCustomer Service:    customer_services@webex.com;    408-435-7088\r\n",
    
    "\r\nJennifer Kouragian\r\nAdministrative Support\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8012\r\nMobile: 408.420-4046\r\nFax: 408.562.8100 \r\njennifer.kouragian@avocadoit.com\r\nwww.avocadoit.com <http://www.avocadoit.com>",
    '\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO\r\nAvocadoIT, Inc.\r\n2211 North First Street, Ste 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile',
    "r\nAmy Moore\r\nExecutive Assistant\r\nMark Tapling, President & CEO\r\nMatt DiMaria, Sr. VP Marketing & Business Development\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8106  fax\r\n(408) 857-8520  mobile",
    "\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd\r\nSanta Clara, CA  95054\r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nThanks,\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO &\r\nExecutive Assistant to Matt DiMaria, Sr. VP of Business Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street, Ste 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile",    
    "\r\nThank you!\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131  \r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nThanks!\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO \r\nand Matt DiMaria, Sr. VP Business Operations\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile",
    "\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131  \r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nA \r\nAmy Moore\r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131  \r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131 \r\ntel: (408) 562-8032\r\nfax: (408) 562-8101\r\nmobile: (408) 857-8520",
    "\r\nThanks,\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO \r\nand Matt DiMaria, Sr. VP Business Operations\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile",
    "\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131  \r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO\r\nand Matt DiMaria, Sr. VP Business Operations\r\nAvocadoIT, Inc.\r\n2211 N. First Street\r\nSuite 200\r\nSan Jose, CA  95131",
    "\r\nThank you,\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO\r\nAvocadoIT, Inc.\r\n2211 North First Street, Ste 200\r\nSan Jose, CA  95131\r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile\r\n",
    "\r\nAmy Moore \r\nExecutive Assistant to Venk Shukla, CEO \r\nAvocadoIT, Inc. \r\n2211 North First Street, Ste 200 \r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel \r\n(408) 562-8108  fax \r\n(408) 857-8520  mobile\r\n",
    "\r\nThanks all,\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO &\r\nExecutive Assistant to Matt DiMaria, Sr. VP of Business Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street, Ste 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile",
    "\r\nAmy Moore \r\nExecutive Assistant to Venk Shukla, CEO \r\nAvocadoIT, Inc. \r\n2211 North First Street, Ste 200 \r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel \r\n(408) 562-8108  fax \r\n(408) 857-8520  mobile \r\n",
    "\r\nThanks,\r\nAmy\r\nAmy Moore \r\nExecutive Assistant to Venk Shukla, CEO \r\nAvocadoIT, Inc. \r\n2211 North First Street, Ste 200 \r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel \r\n(408) 562-8108  fax \r\n(408) 857-8520  mobile",
    "\r\nThanks,\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd\r\nSanta Clara, CA  95054\r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nThanks so much,\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd\r\nSanta Clara, CA  95054\r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nThanks again,\r\nAmy \r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd\r\nSanta Clara, CA  95054\r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nThanks,\r\nNoemi\r\nExecutive Assistant to Venk Shukla",
    "\r\nThanks,\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO\r\nAvocadoIT, Inc.\r\n2211 North First Street, Ste 200\r\nSan Jose, CA  95131\r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile\r\n",
    "\r\nThank you,\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO \r\nand Matt DiMaria, Sr. VP Business Operations\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile",
    "\r\nAmy Moore \r\nExecutive Assistant to Venk Shukla \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA  95131  \r\ntel: (408) 562-8032 \r\nfax: (408) 562-8100",
    "\r\nGermana Martinez\r\nRecruiting Coordinator\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX",
    "\r\nGermana Martinez\r\nHuman Resources\r\nAdministrative Services Coordinator,\r\nAvocadoIT, Inc.\r\n2211 North First St. Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 Phone\r\n408-562-8107 Fax",
    "\r\nGermana Martinez\r\nHuman Resources Coordinator and\r\nSales Executive Admin \r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX",
    "\r\nGermana Martinez\r\nHuman Resources Coordinator,\r\nSales Executive Admin and\r\nAssistant to the CFO\r\nAvocadoIT, Inc.\r\n2211 North First St. Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 Phone\r\n408-562-8107 Fax",
    "\r\nGermana Martinez\r\nRecruiting Coordinator and\r\nExecutive Admin for Sales\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX",
    "\r\nGermana Martinez \r\nHuman Resources Coordinator, \r\nand Executive Admin \r\nAvocadoIT, Inc. \r\n2211 North First St. Suite 200 \r\nSan Jose, CA 95131 \r\n408-562-8135 Phone \r\n408-562-8107 Fax",
    "\r\nGermana Martinez\r\nRecruiting Coordinator\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n408-562-8135 PH\r\n408-562-8100 FX",
    "Germana Martinez\r\nRecruiting Coordinator\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX",
    "Germana Martinez\r\nHuman Resources\r\nAdministrative Services Coordinator,\r\nAvocadoIT, Inc.\r\n2211 North First St. Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 Phone\r\n408-562-8107 Fax",
    "\r\nGermana Martinez \r\nHuman Resources \r\nAdministrative Services Coordinator, \r\nAvocadoIT, Inc. \r\n2211 North First St. Suite 200 \r\nSan Jose, CA 95131 \r\n408-562-8135 Phone \r\n408-562-8107 Fax ",
    "\r\nGermana Martinez\r\nRecruiting Coordinator\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8100 FX",
    "Germana Martinez\r\nRecruiting Coordinator\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n408-562-8135 PH\r\n408-562-8100 FX",
    "Germana Martinez\r\nHuman Resources Coordinator and\r\nSales Executive Admin \r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX",
    "Germana Martinez\r\nHuman Resources Coordinator,\r\nand Executive Admin \r\nAvocadoIT, Inc.\r\n2211 North First St. Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 Phone\r\n408-562-8107 Fax",
    "\r\nGermana Martinez \r\nAdministrative Services Coordinator, \r\nAvocadoIT, Inc. \r\n2211 North First St. Suite 200 \r\nSan Jose, CA 95131 \r\n408-562-8135 Phone \r\n408-562-8107 Fax ",
    "\r\nThanks,\r\nSteven Hirata\r\nDirector of Networks and Operations\r\nAvocadoIT, Inc.\r\nDirect: 408.562.8065 \r\nFax: 408.562.8100 \r\nshirata@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nSteven Hirata\r\nDirector of Networks and Operations\r\nAvocadoIT, Inc.\r\nDirect: 408.562.8065 \r\nFax: 408.562.8100 \r\nshirata@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nSteven Hirata\r\nCEO\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8065 \r\nFax: 408.562.8100 \r\nshirata@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nSteven Hirata\r\nDirector of Networks and Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect - 408-562-8065\r\nshirata@avocadoit.com\r\nhttp://www.avocadoit.com",
    "\r\nRegards,\r\nSteven Hirata\r\nDirector of Networks and Operations\r\nAvocadoIT, Inc.\r\n(408)562-8065",
    "\r\nMike Richman\r\nSenior Account Executive\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice (forwards to cell phone): 415-460-0644",
    "Mike Richman\r\nSenior Account Manager\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice (forwards to cell phone): 415-460-0644\r\nMike.Richman@AvocadoIT.com",
    "\r\nMike Richman\r\nSenior Account Executive\r\n2211 North first Street, Suite 200\r\nSan Jose, CA  95131\r\n415-460-0644 office\r\n408-205-9961 cell\r\n415-460-0722 fax",
    "\r\nMike Richman\r\nSenior Account Executive\r\n2211 North first Street, Suite 200\r\nSan Jose, CA  95131\r\n415-460-0644 office\r\n408-205-6691 cell\r\n415-460-0722 fax",
    "\r\nThanks\r\nStuart\r\nStuart Finn\r\nSenior Vice President, Global Sales and Service\r\nAvocadoIT\r\nstuart.finn@avocadoit.com\r\n408-562-7950",
    "\r\nStuart\r\nStuart Finn\r\nSenior Vice President, Global Sales and Service\r\nAvocadoIT\r\nstuart.finn@avocadoit.com\r\n408-562-7950",
    "\r\nStuart Finn\r\nSenior Vice President, Global Sales and Service\r\nAvocadoIT\r\nstuart.finn@avocadoit.com\r\n408-562-7950",
    "\r\nThanks.\r\nStuart\r\n-------------------------------------------\r\nStuart Finn\r\nSenior Vice President, Global Sales and Service\r\nAvocadoIT, Inc.\r\nstuart.finn@avocadoit.com\r\n408-562-7950\r\n--------------------------------------",
    "\r\nThanks.\r\nStuart\r\nStuart Finn\r\nSenior Vice President, Global Sales & Services\r\nAVOCADOIT, Inc.\r\n2211 North First Street\r\nSan Jose, CA  95131\r\nStuart.Finn@AvocadoIT.com\r\n408-562-7950\r\nwww.AvocadoIT.com",
    "\r\nThanks.\r\nStuart\r\nStuart Finn\r\nSenior Vice President, Global Sales and Service\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\nstuart.finn@avocadoit.com\r\n408-562-7950",
    "\r\nThanks.\r\nStuart Finn\r\nSenior Vice President, Global Sales and Service\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\nstuart.finn@avocadoit.com\r\n408-562-7950",
    "\r\nThanks.\r\nStuart Finn\r\nSenior Vice President,\r\nSales and Service\r\nAVOCADOIT, INC.\r\n2211 N. First Street, Suite 200 \r\nSan Jose, CA 95131 \r\n408-562-7950 (Office) \r\nstuart.finn@avocadoit.com \r\nwww.avocadoit.com",
    '\r\n--------------------------\r\nrm -Rf //taliban/bin/laden',
    '--------------------------\r\nrm -Rf //taliban/bin/laden',
    '\r\n"rm -Rf /bin/laden"\r\nCraig',
    '\r\nCraig',
    '\r\nDoug Field',
    '\r\nDouga....',
    '\r\ndouga...',
    '\r\nJohn',
    '\r\nSrik',
    '\r\nThanks\r\nPrakash',
    '\r\nMahesh',
    '\r\nOm',
    '\r\n-Himmat',
    '\r\nShailesh',
    '\r\nRajeev',
    '\r\nRavi',
    '\r\nSrikanth',
    '\r\nThanks,\r\nSriram',
    '\r\n--Alex',
    '\r\n-Tony',
    '\r\nRayhan\r\n--------------------------\r\nRayhan bin Abdulmughnee\r\nSystems Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7964\r\nMobile: 408.206-8122\r\nFax: 408.562.8100 \r\nrayhan.abdulmughnee@avocadoit.com \r\nwww.avocadoit.com',
    '\r\n--------------------------\r\nRayhan bin Abdulmughnee\r\nSystems Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7964\r\nMobile: 408.206-8122\r\nFax: 408.562.8100 \r\nrayhan.abdulmughnee@avocadoit.com \r\nwww.avocadoit.com',
    "\r\n--------------------------\r\nRayhan bin Abdulmughnee\r\nSystems Engineer\r\nAvocadoIT, Inc.",
    "\r\nDion \r\n----------\r\nDion Bozec, B. Eng\r\nAerospace Engineer\r\nMxi Technologies\r\ndbozec@mxi.com\r\n(613) 747-4698 x266\r\n",
    "\r\nRegards,\r\nTony\r\nAnthony N. Tarsia\r\nDirector of Sales\r\nAvocadoIT, Inc.\r\n4094 Majestic Lane, PMB 127\r\nFairfax, VA 22033\r\nOffice - 703-273-6313      Cell - 703-861-5292\r\nFax - 703-995-4722          E-mail - anthony.tarsia@avocadoit.com\r\nhttp://www.avocadoit.com",
    "\r\nRayhan",
    "\r\nRicardo",
    "\r\nRegards, \r\nVinay Settepalli \r\nSr.Software Engineer, \r\nFrance Telecom R&D,LLC, \r\n1000 Marina Blvd,Suite #300, \r\nBrisbane,CA-94005. \r\n",
    "Rayhan bin Abdulmughnee\r\nSystems Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7964\r\nMobile: 408.206-8122\r\nFax: 408.562.8100 \r\nrayhan.abdulmughnee@avocadoit.com \r\nwww.avocadoit.com",
    "\r\n--------------------------\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n*********************************\r\nBrought to you wirelessly by AvocadoIT Inc.",
    "\r\nJon LaPorte\r\nNW Sales Director\r\nAvocadoIT, Inc.\r\n7041 23rd Avenue NE\r\nSeattle, WA 98115\r\nDirect: 206.985.2040\r\nFax: 309.216.9163\r\njon.laporte@avocadoit.com\r\nwww.avocadoit.com <http://www.avocadoit.com>",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff\r\nVP & Chief Marketing Officer\r\nWe've moved!  Note new phone number:\r\n408.777.4000 ext. 242 talk\r\n408.777.9444 fax\r\nwww.iq.com\r\nmkadanoff@iq.com\r\nWhere insiteful promotions begin\r\n",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\niQ.COM\r\n408.861.9242\r\nwww.iq.com <http://www.iq.com/> \r\nWhere insiteful promotions begin",
    'Marcia Kadanoff\r\nChief Marketing Officer\r\n408.861.9242 direct\r\n408.777.9444 fax\r\nwww.iq.com\r\nWhere insiteful promotions begin\r\n',
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin\r\n\xa0\r\n",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242 talk \r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com> \r\nWhere insiteful promotions begin",
    "\r\n\xa0\r\nMarcia Kadanoff \r\nChief Marketing Officer \r\n408.861.9242 direct \r\n408.777.9444 fax \r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\n--\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\nAvocadoIT\r\n650.270.4309 cell/messages\r\n<<Sent Wirelessly using OmniSky>>\r\n",
    "Marcia Kadanoff\r\nVP & Chief Marketing Officer\r\nWe've moved!  Note new phone number:\r\n408.777.4000 ext. 242 talk\r\n408.777.9444 fax\r\nwww.iq.com\r\nmkadanoff@iq.com\r\nWhere insiteful promotions begin\r\nIn an emergency - page me at mkadanoff.pager@iq.com",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\n408.861.9242 direct\r\n408.777.9444 fax\r\nwww.iq.com\r\nWhere insiteful promotions begin",
    "Marcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\n408.777.4040 fax\r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242 talk                408.777.4040 fax\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com> \r\nWhere insiteful promotions begin",
    "\r\nMK\r\n\xa0\r\n\xa0\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer\r\nNote new direct dial number!\r\n408.861.9242 (talk)\r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer\r\nNote new direct dial number!\r\n408.861.9242 (talk)\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com> \r\nWhere insiteful promotions begin",
    "\r\n\xa0\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 408.777.4040 fax\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer\r\nNote new direct dial number!\r\n408.861.9242 (talk)\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin",
    "Marcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242 talk \r\nmkadanoff@iq.com <mailto:mkadanoff@iq.com> \r\nwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin\r\n",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\niQ.COM\r\n408.861.9242\r\nwww.iq.com <http://www.iq.com/>\r\nWhere insiteful promotions begin\r\n",
    "\r\n\xa0\r\nMK\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\niQ.COM\r\n408.861.9242\r\nwww.iq.com <http://www.iq.com/>\r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\nAvocadoIT\r\n408.562.8050 direct\r\n650.270.4309 mobile",
    "\r\n\xa0\r\nBest regards,\r\n\xa0\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\niQ.COM\r\n408.861.9242\r\nwww.iq.com\r\nWhere insiteful promotions begin",
    "\r\n--\r\nMarcia Kadanoff\r\n<<Sent Wirelessly using OmniSky>>\r\n",
    "iQ.COM\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "Marcia Kadanoff \r\nVP & Chief Marketing Officer\r\nNote new direct dial number!\r\n408.861.9242 (talk)\r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff\r\nVP & Chief Marketing Officer\r\niQ.COM\r\n408.777.4000 ext. 242 talk\r\n408.777.9444 fax\r\nwww.iq.com\r\nWhere insiteful promotions begin",
    "Marcia Kadanoff\r\nVP & Chief Marketing Officer\r\nWe've moved!  Note new phone number:\r\n408.777.4000 ext. 242 talk\r\n408.777.9444 fax\r\nwww.iq.com\r\nmkadanoff@iq.com\r\nWhere insiteful promotions begin",
    "Marcia Kadanoff\r\nChief Marketing Officer\r\n408.861.9242 direct\r\n408.777.9444 fax\r\nwww.iq.com\r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff \r\nChief Marketing Officer \r\nAvocadoIT \r\n408.562.8050 - new direct dial #! \r\nwww.avocadoit.com",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\nAvocadoIT\r\n408.562.8050 - new direct dial #!\r\nwww.avocadoit.com",
    "Marcia Kadanoff \r\nVP & Chief Marketing Officer\r\nNote new direct dial number!\r\n408.861.9242 (talk)\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin",
    "\r\n\xa0\r\nMK\r\nMarcia Kadanoff \r\nChief Marketing Officer \r\n408.861.9242 direct \r\n408.777.9444 fax \r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\n\xa0\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\niQ.COM\r\n408.861.9242\r\nwww.iq.com\r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff\r\n650.343.4309 talk\r\n650.343.2042 fax\r\nmarciak@msdirect.com\r\nwww.msdirect.com\r\n",
    "\r\n\xa0\r\nmk\r\nMarcia Kadanoff \r\nChief Marketing Officer \r\n408.861.9242 direct \r\n408.777.9444 fax \r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nRitesh Adval\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000 \r\nDirect: 408.562.8035 \r\nFax: 408.562.8100 \r\nradval@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nRitesh Adval\r\nAvocadoIT, Inc.\r\n2211 North First Street, \r\nSuite 200, San Jose,\r\nCA  95131\r\nDirect: 408.562.8035 \r\nFax: 408.562.8103\r\nradval@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nRitesh Adval\r\nSoftware Engineer\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000 \r\nDirect: 408.562.8035 \r\nFax: 408.562.8100 \r\nradval@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nThanks\r\nRitesh\r\nRitesh Adval\r\nAvocadoIT, Inc.\r\n2211 North First Street,\r\nSuite 200, San Jose,\r\nCA  95131\r\nDirect: 408.562.8035\r\nFax: 408.562.8103\r\nradval@avocadoit.com\r\nwww.avocadoit.com\r\n",
    "\r\nLisa Zissen  \r\nExecutive Assistant to Prakash Iyer\r\nFounder, CTO and Sr. VP of Engineering Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\n2nd Floor\r\nSan Jose, California  95131\r\n(408) 562-8012 - direct\r\n(408) 562-8104 - Fax\r\nlisa.zissen@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nLisa Zissen\r\nExec. Asst. to Prakash Iyer, CTO & Founder\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\n2nd Floor\r\nSan Jose, California  95131\r\n(408) 562-8012 - direct\r\n(408) 562-8104 - Fax\r\nlisa.zissen@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nThanks!\r\nLisa\r\nLisa Zissen\r\nExecutive Assistant to Prakash Iyer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite #200\r\nSan Jose, California  95131\r\n408.562.8012 - direct\r\n408.205.0665 - Cell\r\nlisa.zissen@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nLisa Zissen \r\nExecutive Assistant to Prakash Iyer \r\nFounder, CTO and Sr. VP of Engineering Operations \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\n2nd Floor \r\nSan Jose, California  95131 \r\n(408) 562-8012 - direct \r\n(408) 205-0665 - Mobile \r\n(408) 562-8100 - Fax \r\nlisa.zissen@avocadoit.com \r\nwww.avocadoit.com \r\nAvocadoIT, your business everywhere! \r\n_____  \r\nUpgrade Your Email - Click here!",
    "\r\nRay Rahamin\r\nSent Wirelessly form AvocadoIT -- The Leading Enterprise Mobility Company",
    "\r\nRay Rahamin\r\nGeneral Manager\r\nFinancial Services Industry\r\nSent Wirelessly from AvocadoIT -- The Leading Enterprise Mobility Company",
    "\r\nRay Rahamin\r\nGeneral Manager\r\nFinancial Services Industry\r\nAvocadoIT\r\nOffice: 914-773-0593\r\nCell   : 914-645-6617\r\nFax  : 914-773-0595\r\nRay.Rahamin@avocadoit.com",
    "\r\nRay Rahamin\r\nGeneral Manager\r\nFinancial Services Industry!",
    "\r\nThanks, \r\nMehrak Hamzeh\r\nMobility Consultant\r\nAvocadoIT, Inc.\r\n2211 North First Street Suite 200\r\nSan Jose, CA  95131\r\nTelephone: 408.562.7989\r\nFax: 408.562.8100 \r\nhttp://www.avocadoit.com",
    '\r\nMehrak Hamzeh\r\nMobility Consultant\r\nAvocadoIT, Inc.\r\n2211 North First Street Suite 200\r\nSan Jose, CA  95131\r\nTelephone: 408.562.7989\r\nFax: 408.562.8100\r\nAre you ready for the wireless web? \r\nhttp://www.avocadoit.com',
    "\r\nMehrak Hamzeh\r\nMobility Consultant\r\nAvocadoIT, Inc.\r\n2211 North First Street Suite 200\r\nSan Jose, CA  95131\r\nTelephone: 408.562.7989\r\nFax: 408.562.8100 \r\nhttp://www.avocadoit.com",
    "\r\nMehrak Hamzeh\r\nMobility Consultant\r\nwww.avocadoit.com\r\n3231 Scott Blvd. Santa Clara, CA 95054\r\nPhone: 408.562.7989\r\nFax:408.562.8100",
    "\r\nRay Rahamin\r\nGeneral Manager of Financial Services Industry \r\nOffice:  914.773.0593\r\nCell:      914.645.6617\r\nfax:  914.773.0595",
    "\r\nRyan Glover \r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8057 office\r\n408.562.8100 fax\r\n408.420.3841 mobile\r\nryan.glover@avocadoit.com\r\nhttp://www.avocadoit.com",
    "\r\nAjay Karwal\r\nakarwal@avocadoit.com\r\nwww.avocadoit.com\r\n408-562-8111",
    "\r\nThanks,\r\najay\r\nAjay Karwal\r\nApplication Architect\r\nakarwal@avocadoit.com\r\nwww.avocadoit.com\r\n408-562-8111(Direct)\r\n408-420-3854(Cell)",
    "\r\nAjay Karwal\r\nManager, Application Engineering\r\nakarwal@avocadoit.com\r\nwww.avocadoit.com\r\n408-562-8111(Direct)",
    "McCann\r\nAvocadoIT, Inc\r\n8745 W. Higgins, Suite 360\r\nChicago, IL 60631\r\nOffice:  773.243-0324\r\nMobile: 847-826-8064\r\nFax: 773-243-0225.McCannl@avocadoit.com\r\nwww.avocadoit.com\r\n--------------------------\r\nSent from my Wireless Handheld",
    "McCann\r\nAvocadoIT, Inc\r\n408-562-7951\r\n--------------------------\r\nSent from my Wireless Handheld",
    "\r\nThanks,.McCann@avocadoit.com\r\nAvocadoIT, Inc.\r\n408-562-7951",
    "McCann, Central Region Director\r\nAvocadoIT, Inc\r\nOffice:  773.243-0324\r\nMobile: 847-826-8064\r\nFax: 773-243-0225.McCann@avocadoit.com\r\nwww.avocadoit.com\r\n---------------------",
    'McCann, District Manager\r\nAvocadoIT, Inc.\r\n8735 W. Higgins, Suite 235\r\nChicago, IL 60631\r\nOffice 773-243-0324\r\nFax  773-243-0225\r\nwww.AvocadoIT.com\r\n"AvocadoIT, Your Business Everywhere!"',
    "\r\nNihar Mehta\r\nAvocadoIT Inc.\r\n408-562-8019",
    "\r\n-------------------------------\r\nSent Wirelessly from AvocadoIT - the Leading Enterprise Mobile Company",
    "\r\n________________________\r\nChris Longstaffe\r\nDirector of Training\r\nAvocadoIT, Inc\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8132\r\nMobile: 408.205.7573\r\nFax: 408.562.8014\r\nchris.longstaffe@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nChris\r\n--------------------------\r\nChris Longstaffe\r\n(408) 205 7573\r\nSent using my AvocadoIT wireless device\r\nAvocadoIT ... Your Business, Everywhere!",
    "\r\nThanks\r\nChris\r\n________________________\r\nChris Longstaffe\r\nDirector of Training, AvocadoIT\r\nTel: (408) 562 8132\r\nCell: (925) 640 3166",
    "\r\nthanks\r\nChris\r\n_______________________\r\nChris Longstaffe\r\nSenior Director, Education Services\r\nAvocadoIT, Inc\r\n2211 North First St, Suite 200\r\nSan Jose, CA 95131\r\nDirect: (408) 562 8132\r\nMobile: (408) 205 7573\r\nFax: (408) 562 8104\r\nchris.longstaffe@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks\r\nChris\r\n________________________\r\nChris Longstaffe\r\nDirector of Training, AvocadoIT\r\nTel: (408) 562 8132\r\nCell: (408) 205 7573",
    "\r\nChris\r\n________________________\r\nChris Longstaffe\r\nSenior Director, Education Services\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSan Jose, CA 95131\r\nOffice: (408) 562 8132\r\nMobile: (408) 205 7573",
    "\r\nThanks\r\nChris\r\n________________________\r\nChris Longstaffe\r\nSenior Director, Education Services\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSan Jose, CA 95131\r\nOffice: (408) 562 8132\r\nMobile: (408) 205 7573",
    "\r\nChris\r\n________________________\r\nChris Longstaffe\r\nSenior Director, Education Services\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSan Jose, CA 95131\r\nOffice: (408) 562 8132\r\nMobile: (408) 205 7573",
    "\r\nThanks\r\nChris\r\n_______________________\r\nChris Longstaffe\r\nSenior Director, Education Services\r\nAvocadoIT, Inc\r\n2211 North First St, Suite 200\r\nSan Jose, CA 95131\r\nDirect: (408) 562 8132\r\nMobile: (408) 205 7573\r\nFax: (408) 562 8104\r\nchris.longstaffe@avocadoit.com\r\nwww.avocadoit.com",
    "\r\n_______________________\r\nChris Longstaffe\r\nSenior Director, Education Services\r\nAvocadoIT, Inc\r\n2211 North First St, Suite 200\r\nSan Jose, CA 95131\r\nDirect: (408) 562 8132\r\nMobile: (408) 205 7573\r\nFax: (408) 562 8104\r\nchris.longstaffe@avocadoit.com\r\nwww.avocadoit.com",
    "\r\n--------------------------\r\nChris Longstaffe\r\n(408) 205 7573\r\nSent using my AvocadoIT wireless device\r\nAvocadoIT ... Your Business, Everywhere!",
    "\r\nChris\r\n________________________\r\nChris Longstaffe\r\nMobile: (925) 640 3166\r\nchris@integratis.com",
    "Thanks.\r\nChris Longstaffe\r\n(408)205 7573",
    "Scott\r\n--------------------------\r\nSent Wirelessly from AvocadoIT - the leading enterprise mobility company\r\nScott Weller\r\n206-323-5105 (o)\r\n206-604-6105 (m)",
    "\r\nScott Weller\r\nSenior Director - Telecom\r\nAvocadoIT, Inc\r\n206-323-5105 (o)\r\n206-604-6105 (m)\r\n206-323-4314 (f)\r\nwww.avocadoit.com",
    "\r\nScott\r\nScott Weller\r\nDirector, Business Development - Telecom\r\nAvocadoIT, Inc\r\n206-323-5105 (o)\r\n206-604-6105 (m)\r\n206-323-4314 (f)\r\nwww.avocadoit.com",
    "\r\nScott Weller\r\nDirector, Business Development - Telecom\r\nAvocadoIT, Inc\r\n206-323-5105 (o)\r\n206-604-6105 (m)\r\n206-323-4314 (f)\r\nwww.avocadoit.com",
    "\r\nThanks,\r\nScott\r\nScott Weller\r\nSr. Director - Telecom\r\nAvocadoIT, Inc\r\n(o)  206.323.5105\r\n(m)  206.604.6105\r\n(f)  206.323.4314",
    "\r\nRegards,\r\nScott\r\nScott Weller\r\nSr. Director - Telecom\r\nAvocadoIT, Inc\r\n(o)  206.323.5105\r\n(m) 206.604.6105\r\n(f)   206.323.4314",
    "\r\nScott\r\nScott Weller\r\nSr. Director - Telecom\r\nAvocadoIT, Inc\r\n(o)  206.323.5105\r\n(m)  206.604.6105\r\n(f)  206.323.4314",
    "\r\nKhou Fang\r\nTechnical Recruiting Manager\r\nDirect: 408.562.8096\r\nMobile: 408.829-4796\r\nFax: 408.562.8107",
    "\r\nScott Weller\r\nDirector - Telecom Business Development\r\nAvocadoIT\r\n206-323-5105 (o)\r\n206-604-6105 (m)\r\nhttp://www.avocadoit.com",

    ]  # Regex patterns

for item in removal_list:
    texts = texts.str.replace(item,"")


0         Bug 5539\r\nThe screen wizard now ignores the ...
1         At 02:50 PM 3/15/2001 +0100, it seemed like Ma...
2         All,\r\nWe are still in a Nor'easter, but the ...
3         Responding to Robert Sartin (07:02 AM 6/14/200...
4         I will try to get the real secure up at the fi...
                                ...                        
128640    Checked in WinInetBrowser.java to fix a bug in...
128641    Just wanted to make sure that you know I did t...
128642    In the end, you will always fail to think of e...
128643    Yes, it's valid for Europe, as well.  Venk for...
128644    Hey John,\r\nThe Siebel Adapter will allow you...
Name: text, Length: 128641, dtype: object

## Save to DF

In [11]:
df = texts.to_frame()
df.to_parquet('../../data/processed/05_avocado_cleaned_clustered_wip_7.parquet', engine='pyarrow')

In [ ]:
len(df)